# Mini-Projet Intelligence Artificielle : Prédiction du Revenu Annuel d'un Marocain

**Membres du Groupe :**
*   Saad Barhrouj
*   Nassim El Kaddaoui
*   Youness Kihel

**Année :** 2ème année Cycle d’Ingénieurs – GI
**Module :** Intelligence Artificielle
**Encadrant :** Y. EL YOUNOUSSI
**Année Universitaire :** 2024-2025
**Date de présentation :** 17 mai 2025 

---

*Ce notebook présente notre travail pour le mini-projet d'Intelligence Artificielle. Il détaille chaque étape de la construction d'un modèle de prédiction du revenu annuel des Marocains, depuis la génération des données jusqu'au déploiement.*

L'objectif général de ce mini-projet est de construire un pipeline complet de Machine Learning en Python pour prédire le revenu annuel des Marocains à partir de données simulées réalistes. Le projet couvre toutes les étapes de développement d’un modèle de Machine Learning : compréhension des données, préparation des données, modélisation, évaluation et déploiement.

# 0. Génération du Dataset Synthétique

Avant toute analyse ou modélisation, la première étape cruciale de ce projet a été la création d'un dataset synthétique. Ce dataset, nommé dataset_revenu_marocains.csv, a été conçu pour simuler de manière réaliste les revenus des Marocains et leurs facteurs d'influence, en s'inspirant des statistiques du Haut-Commissariat au Plan (HCP) et des exigences du projet. Le script Python generate_dataset.py est l'outil central de cette génération.

**Objectif du script generate_dataset.py :**

L'objectif principal est de produire un fichier CSV structuré, contenant environ 40 000 enregistrements. Chaque enregistrement représente un individu fictif avec diverses caractéristiques socio-économiques et son revenu annuel. Le script vise à créer un équilibre entre réalisme statistique et introduction contrôlée d'imperfections pour simuler les défis rencontrés avec des données réelles.

**Paramètres initiaux et reproductibilité :**

Au début du script, des paramètres clés sont définis :

```python
# ...existing code...
# --- Paramètres Généraux ---
N_RECORDS = 40000
FILENAME = "dataset_revenu_marocains.csv"
RANDOM_SEED = 42 # Pour la reproductibilité
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# --- Définition des Catégories ---
MILIEU_OPTS = ['Urbain', 'Rural']
SEXE_OPTS = ['Homme', 'Femme']
NIVEAU_EDUCATION_OPTS = ['Sans niveau', 'Fondamental', 'Secondaire', 'Supérieur']
# ...autres options...
# ...existing code...
```

*   `N_RECORDS = 40000` : Définit le nombre d'individus (enregistrements) à générer.
*   `FILENAME = "dataset_revenu_marocains.csv"` : Spécifie le nom du fichier de sortie.
*   `RANDOM_SEED = 42` : Est utilisé pour initialiser les générateurs de nombres aléatoires de NumPy et Python. Cela garantit que si le script est exécuté plusieurs fois avec la même graine, le dataset généré sera identique, assurant ainsi la reproductibilité des résultats.
*   Des listes comme `MILIEU_OPTS`, `SEXE_OPTS`, etc., définissent les valeurs possibles pour les variables catégorielles.

## Logique de Génération des Caractéristiques

Le script génère les caractéristiques de manière séquentielle, certaines étant indépendantes et d'autres dépendant des valeurs précédemment générées pour simuler des corrélations logiques.

### 1. Caractéristiques de Base

Ces caractéristiques sont souvent générées en premier, en utilisant des distributions de probabilités prédéfinies.

*   **Âge (`generate_age`) :** Généré aléatoirement entre 18 et 63 ans.
    ```python
    # filepath: /Users/youns/Desktop/Projects/ML/generate_dataset.py
    # ...existing code...
    def generate_age(n):
        return np.random.randint(18, 64, n)
    # ...existing code...
    ```
*   **Milieu (`generate_milieu`) :** 'Urbain' ou 'Rural', avec une probabilité cible (ex: 60% Urbain).
    ```python
    # filepath: /Users/youns/Desktop/Projects/ML/generate_dataset.py
    # ...existing code...
    P_URBAIN = 0.60 # Proportion cible d'urbains (ajustable)
    P_RURAL = 1 - P_URBAIN
    # ...
    def generate_milieu(n):
        return np.random.choice(MILIEU_OPTS, n, p=[P_URBAIN, P_RURAL])
    # ...existing code...
    ```
*   **Sexe (`generate_sexe`) :** 'Homme' ou 'Femme', avec des probabilités légèrement ajustées (ex: 52% Homme).
*   **Niveau d'éducation (`generate_niveau_education`) :** Choisi parmi 'Sans niveau', 'Fondamental', 'Secondaire', 'Supérieur', avec des probabilités reflétant une certaine répartition.

### 2. Caractéristiques Dépendantes

Ces caractéristiques sont générées en tenant compte d'autres attributs pour créer des relations plausibles.

*   **Années d’expérience (`generate_annees_experience`) :** Dépend de l'`Age` et du `Niveau_education`. L'âge minimum pour commencer à travailler varie selon le niveau d'éducation, et l'expérience ne peut excéder l'âge moins un âge minimum d'entrée sur le marché du travail.
    ```python
    # filepath: /Users/youns/Desktop/Projects/ML/generate_dataset.py
    # ...existing code...
    def generate_annees_experience(age, niveau_education):
        experience = []
        for a, edu in zip(age, niveau_education):
            min_age_travail = 18
            if edu == 'Supérieur': min_age_travail = 23
            elif edu == 'Secondaire': min_age_travail = 20
            
            max_exp = a - min_age_travail
            if max_exp < 0: max_exp = 0
            
            exp = np.random.randint(0, max_exp + 1) if max_exp > 0 else 0
            exp = min(exp, a - 16) if a > 16 else 0 # Assure que l'expérience n'est pas illogique par rapport à l'âge
            experience.append(max(0,exp))
        return np.array(experience)
    # ...existing code...
    ```
*   **État matrimonial (`generate_etat_matrimonial`) :** Influencé par l'`Age`. Les probabilités des différents états matrimoniaux changent selon que l'individu est jeune, adulte ou plus âgé.
*   **Catégorie Socioprofessionnelle (CSP) (`generate_csp`) :** Déterminée par le `Niveau_education`, les `Annees_experience`, et l'`Age`. Par exemple, un niveau d'éducation 'Supérieur' avec beaucoup d'expérience mènera plus probablement à 'Cadres supérieurs'.
*   **Possession de biens (`generate_possession_biens`) :** (Propriété immobilière, Véhicule motorisé, Terrain agricole). La probabilité de posséder ces biens est liée à la `CSP` et au `Milieu`. Par exemple, les 'Cadres supérieurs' ont une plus forte probabilité de posséder un bien immobilier, et les 'Agriculteurs' en milieu 'Rural' une plus forte probabilité de posséder un terrain agricole.

### 3. Caractéristiques Additionnelles

Ces caractéristiques ajoutent plus de granularité au dataset.

*   **Région géographique (`generate_region_geographique`) :** Choisie parmi 'Nord', 'Centre', 'Sud', 'Est', 'Ouest', avec des probabilités visant à refléter des pôles économiques.
*   **Secteur d'emploi (`generate_secteur_emploi`) :** ('Public', 'Privé', 'Informel'). Dépend de la `CSP`. Les 'Inactifs' n'ont pas de secteur d'emploi (NaN), tandis que les 'Agriculteurs' sont plus susceptibles d'être dans le 'Privé' ou 'Informel'.
*   **Revenu secondaire (`generate_revenu_secondaire`) :** Booléen ('Oui'/'Non'). La probabilité d'avoir un revenu secondaire est plus élevée pour certaines `CSP` comme 'Cadres supérieurs'.

### 4. Génération du Revenu Annuel (`generate_revenu_annuel`)

C'est la partie la plus complexe, visant à respecter les contraintes statistiques du HCP.

```python
# ...existing code...
def generate_revenu_annuel(df):
    n = len(df)
    revenus = np.zeros(n)
    
    is_urbain = (df['Milieu'] == 'Urbain').values
    is_rural = (df['Milieu'] == 'Rural').values

    # 1. Génération de base (Log-normale)
    base_revenu_log_mean_urbain = np.log(11000) 
    base_revenu_log_sigma_urbain = 0.65 
    base_revenu_log_mean_rural = np.log(7000)
    base_revenu_log_sigma_rural = 0.95  

    if is_urbain.sum() > 0:
        revenus[is_urbain] = np.random.lognormal(mean=base_revenu_log_mean_urbain,
                                                  sigma=base_revenu_log_sigma_urbain,
                                                  size=is_urbain.sum())
    if is_rural.sum() > 0:
        revenus[is_rural] = np.random.lognormal(mean=base_revenu_log_mean_rural,
                                                 sigma=base_revenu_log_sigma_rural,
                                                 size=is_rural.sum())

    # 2. Application des bonus/facteurs
    for i in range(n):
        record = df.iloc[i]
        rev = revenus[i]
        # Ex: Niveau d'éducation
        if record['Niveau_education'] == 'Supérieur':
            rev += 25000 * (1 + 0.15 * record['Annees_experience']) 
        # ... autres facteurs (CSP, Sexe, Région, etc.) ...
        revenus[i] = rev

    # 3. Calibration des moyennes par milieu
    CIBLE_URBAIN = 26988
    CIBLE_RURAL = 12862
    mean_urbain = df_temp.loc[is_urbain, 'rev'].mean() if is_urbain.sum() > 0 else 1
    # ... calibration ...
    if is_urbain.sum() > 0 and mean_urbain > 0 : 
        revenus[is_urbain] *= CIBLE_URBAIN / mean_urbain
    # ... idem pour rural ...

    # 4. Plafonnement initial
    revenus = np.clip(revenus, 300, 600000)
    # ... traitement spécifique pour les inactifs ...

    # 5. Ajustement des proportions sous la moyenne (avec adjust_group_revenus)
    global_target = 0.718
    # ...
    if revenus.size > 0 :
        revenus = adjust_group_revenus(revenus, global_target) # adjust_group_revenus utilise une transformation puissance
    # ... idem pour urbain et rural ...

    # 6. Renormalisation des moyennes et plafonnement final
    # ...
    return revenus
# ...existing code...
```

**Étapes clés de la génération du revenu :**

1.  **Distribution de base :** Le revenu initial est tiré d'une distribution log-normale, avec des paramètres (moyenne et écart-type sur l'échelle logarithmique) différents pour les milieux urbain et rural. La distribution log-normale est choisie car elle modélise bien les quantités qui sont le produit de nombreux petits facteurs et qui sont positivement asymétriques, comme les revenus.
2.  **Ajustements par facteurs :** Ce revenu de base est ensuite modifié (augmenté ou diminué) par des bonus additifs et des facteurs multiplicatifs basés sur les autres caractéristiques de l'individu :
    *   `Niveau_education` et `Annees_experience` : Un niveau d'éducation plus élevé et plus d'expérience augmentent significativement le revenu.
    *   `CSP` : Chaque CSP a un facteur multiplicatif (ex: 'Cadres supérieurs' ont un facteur plus élevé).
    *   `Sexe` : Un léger ajustement est fait (ex: hommes gagnent un peu plus en moyenne).
    *   `Region_geographique` : Certaines régions (pôles économiques) ont un facteur positif.
    *   `Secteur_emploi` : Le secteur (Public, Privé, Informel) influence le revenu.
    *   `Revenu_secondaire` : Ajoute un montant fixe si 'Oui'.
3.  **Calibration des moyennes :** Après l'application des facteurs, les revenus moyens pour les milieux urbain et rural sont calculés. Ils sont ensuite recalibrés (multipliés par un ratio) pour correspondre aux cibles du HCP (26 988 DH pour l'urbain, 12 862 DH pour le rural).
4.  **Plafonnement :** Les revenus sont bornés (ex: entre 300 DH et 600 000 DH) pour éviter des valeurs extrêmes irréalistes. Un traitement spécifique est appliqué aux 'Inactifs' pour limiter leur revenu maximal.
5.  **Ajustement des proportions :** La fonction `adjust_group_revenus` est cruciale. Elle applique une transformation de puissance aux revenus pour s'assurer que la proportion d'individus ayant un revenu inférieur à la moyenne (globale, urbaine, rurale) se rapproche des cibles du HCP (ex: 71,8% globalement). Cette transformation modifie la forme de la distribution tout en préservant l'ordre des revenus.
6.  **Renormalisation et Plafonnement Final :** Après l'ajustement des proportions, les moyennes peuvent avoir légèrement bougé. Elles sont donc recalculées et les revenus sont à nouveau normalisés pour atteindre les moyennes cibles. Un dernier plafonnement est appliqué.

### 5. Introduction des Imperfections

Pour rendre le dataset plus réaliste et stimulant pour l'analyse, des imperfections sont délibérément introduites.

*   **Valeurs Manquantes (`add_valeurs_manquantes`) :** Des `NaN` sont insérés aléatoirement dans certaines colonnes (ex: `Etat_matrimonial`, `Secteur_emploi`, `Annees_experience`) avec une faible probabilité.
    ```python
    # filepath: /Users/youns/Desktop/Projects/ML/generate_dataset.py
    # ...existing code...
    def add_valeurs_manquantes(df, colonnes_pour_nan, p_nan=0.001):
        for col in colonnes_pour_nan:
            if col in df.columns:
                mask = np.random.choice([True, False], size=len(df), p=[p_nan, 1-p_nan])
                df.loc[mask, col] = np.nan
        return df
    # ...existing code...
    ```
*   **Valeurs Aberrantes :**
    *   `add_valeurs_aberrantes_age` : Introduit des âges illogiques (ex: -5, 150).
    *   `add_valeurs_aberrantes_experience` : Crée des cas où l'expérience est supérieure à l'âge.
    *   `add_valeurs_aberrantes_possession` : Simule des situations illogiques (ex: un 'Inactif' possédant de nombreux biens, ou un 'Agriculteur' sans terrain agricole).
    *   `add_valeurs_aberrantes_revenu` : Ajoute des revenus extrêmes ou incohérents avec la `CSP` (ex: revenu très bas pour un 'Cadre supérieur' ou très élevé pour un 'Inactif').
        ```python
        # filepath: /Users/youns/Desktop/Projects/ML/generate_dataset.py
        # ...existing code...
        def add_valeurs_aberrantes_revenu(revenus_col, csp_col, p_aberrant=0.0005):
            n_aberr = int(p_aberrant * len(revenus_col))
            # ...
            for idx in aberr_indices:
                csp = csp_col.loc[idx]
                if csp == 'Inactifs':
                    revenus_col.loc[idx] = np.random.choice([100, 200, 300])
                # ... autres CSP ...
            return revenus_col
        # ...existing code...
        ```
*   **Colonnes Redondantes :**
    *   `Revenu_Mensuel` : Simplement `Revenu_Annuel / 12`.
    *   `Categorie_age` : Dérivée de la variable `Age` ('Jeune', 'Adulte', 'Senior', 'Âgé').
*   **Colonnes Non Pertinentes :**
    *   `Adresse_Email` : Générée avec un format d'email unique.
    *   `CIN` : Un identifiant unique simulé.

### 6. Assemblage Final et Sauvegarde

Toutes ces caractéristiques générées et modifiées sont assemblées dans un DataFrame Pandas. Des vérifications finales de cohérence sont effectuées (ex: s'assurer que l'expérience n'est pas supérieure à l'âge après l'introduction d'aberrations, ou que les inactifs ont des revenus bas).
Enfin, l'ordre des colonnes est défini et le DataFrame est sauvegardé dans le fichier CSV spécifié.

```python
# ...existing code...
# --- Main Execution ---
if __name__ == "__main__":
    print(f"Génération de {N_RECORDS} enregistrements avec le script ...")
    dataset = generate_dataset() # Appelle la fonction principale qui orchestre tout
    
    # ... (impressions des statistiques et vérifications) ...

    dataset.to_csv(FILENAME, index=False, encoding='utf-8')
    print(f"\nDataset '{FILENAME}' généré avec succès ({len(dataset)} enregistrements).")
# ...existing code...
```

En résumé, le script generate_dataset.py est un outil sophistiqué qui construit un dataset riche et complexe, en s'efforçant de respecter des contraintes statistiques tout en introduisant des éléments qui miment les défis des données du monde réel. Ce dataset sert de fondation solide pour toutes les étapes ultérieures du projet de Machine Learning.

# 1. Compréhension des Données

Après avoir résumé le processus de génération du dataset, nous allons maintenant charger le fichier `dataset_revenu_marocains.csv` et l'explorer en détail. L'objectif est de bien comprendre sa structure, son contenu, et d'identifier les caractéristiques notables avant de passer à la préparation des données pour la modélisation.

In [108]:
# --- 1. IMPORTATION DES LIBRAIRIES ---
# On charge ici les outils nécessaires pour notre analyse (manipulation de données, maths, graphiques).
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- 2. CONFIGURATION DE LA REPRODUCTIBILITÉ ---
# On fixe une "graine" pour le hasard, pour que nos résultats soient toujours les mêmes si on relance.
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# --- 3. OPTIONS D'AFFICHAGE PANDAS ---
# On règle comment les tableaux de données (DataFrames) vont s'afficher à l'écran.
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 100) 
pd.set_option('display.width', 1000) 

# --- 4. STYLE DES GRAPHIQUES ---
# On choisit un style visuel pour rendre nos graphiques plus agréables à regarder.
plt.style.use('seaborn-v0_8-whitegrid') 
sns.set_palette("pastel")             

# --- 5. GESTION DES MESSAGES D'AVERTISSEMENT ---
# On demande à Python de ne pas afficher certains messages techniques (warnings) pour un rendu plus propre.
import warnings
warnings.filterwarnings('ignore')

# --- 6. VÉRIFICATION ET INFORMATIONS ---
# On affiche un message pour confirmer que tout est prêt et les versions des outils utilisés.
print("Librairies pour l'analyse de données importées.")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Seaborn version: {sns.__version__}")

Librairies pour l'analyse de données importées.
Pandas version: 2.2.3
Numpy version: 2.1.0
Seaborn version: 0.13.2


## 1.1 Chargement du Dataset
Nous chargeons le fichier `dataset_revenu_marocains.csv` qui a été généré par notre script `generate_dataset.py`. Ce fichier contient les données simulées sur lesquelles nous allons travailler.

In [109]:
# --- 1. DÉFINITION DU NOM DU FICHIER ET INITIALISATION ---
# On indique le nom du fichier de données à charger et on prépare une variable pour le contenir.
FILENAME = "dataset_revenu_marocains.csv"
df = None                                

# --- 2. CHARGEMENT DES DONNÉES AVEC GESTION DES ERREURS ---
# On essaie de lire le fichier. Si ça ne marche pas, on affiche un message clair au lieu de faire planter le programme.
try:
    # Tentative de lecture du fichier CSV avec pandas.
    df = pd.read_csv(FILENAME)
    print(f"Le fichier '{FILENAME}' a été chargé avec succès dans le DataFrame 'df'.")
    print(f"Dimensions du DataFrame : {df.shape[0]} lignes et {df.shape[1]} colonnes.") # df.shape donne (lignes, colonnes)

except FileNotFoundError:
    # Si le fichier n'est pas trouvé à l'endroit attendu :
    print(f"ERREUR CRITIQUE : Le fichier '{FILENAME}' n'a pas été trouvé.")
    print("Veuillez vérifier que le script de génération a bien été exécuté et que le fichier CSV est au bon emplacement (généralement le même dossier que ce notebook).")
    print("Les étapes suivantes ne pourront pas être exécutées sans le chargement des données.")

except Exception as e:
    # Si une autre erreur se produit pendant le chargement (ex: fichier corrompu, mauvais format) :
    print(f"Une erreur est survenue lors du chargement du fichier '{FILENAME}' :")
    print(e) 
    print("Les étapes suivantes ne pourront pas être exécutées sans le chargement des données.")

# --- 3. VÉRIFICATION FINALE DU CHARGEMENT ---
# On vérifie si les données ont bien été chargées dans 'df' avant de continuer.
if df is not None:
    # Si 'df' n'est plus vide (donc le chargement a réussi) :
    print("\nLe DataFrame 'df' est prêt pour l'analyse.")
else:
    # Si 'df' est toujours vide (le chargement a échoué) :
    print("\nATTENTION : Le DataFrame 'df' n'a pas été chargé. Veuillez corriger le problème avant de continuer.")

Le fichier 'dataset_revenu_marocains.csv' a été chargé avec succès dans le DataFrame 'df'.
Dimensions du DataFrame : 40000 lignes et 18 colonnes.

Le DataFrame 'df' est prêt pour l'analyse.


## 1.2 Affichage des Données (Aperçu)
Pour avoir une première idée concrète du contenu et de la structure des données, affichons les 10 premières et les 10 dernières lignes du DataFrame. Cela nous permet de vérifier rapidement les noms des colonnes et le format des données.

In [110]:
# --- VÉRIFICATION DU CHARGEMENT AVANT AFFICHAGE ---
# On s'assure que le DataFrame 'df' contient bien des données avant d'essayer de les montrer.
if df is not None:
    # Si 'df' a été chargé correctement (il n'est pas vide) :

    # --- 1. AFFICHAGE DES PREMIÈRES LIGNES ---
    # On regarde le début du tableau pour avoir une première idée des données.
    print("Aperçu des 10 premières lignes du DataFrame :")
    display(df.head(10))

    # --- 2. AFFICHAGE DES DERNIÈRES LIGNES ---
    # On regarde aussi la fin du tableau, parfois utile pour vérifier l'intégrité des données ou des totaux.
    print("\nAperçu des 10 dernières lignes du DataFrame :")
    display(df.tail(10)) 

else:
    # --- GESTION DU CAS OÙ LES DONNÉES NE SONT PAS CHARGÉES ---
    # Si 'df' est vide (le chargement a échoué à l'étape précédente) :
    print("ATTENTION : Le DataFrame 'df' n'est pas chargé. Impossible d'afficher les données.")

Aperçu des 10 premières lignes du DataFrame :


,Age,Categorie_age,Sexe,Milieu,Region_geographique,Etat_matrimonial,Niveau_education,Annees_experience,CSP,Secteur_emploi,Propriete_immobiliere,Vehicule_motorise,Terrain_agricole,Revenu_secondaire,Revenu_Annuel,Revenu_Mensuel,Adresse_Email,CIN
0,56,Senior,Homme,Rural,Centre,Marié,Fondamental,5.0,Agriculteurs,Privé,Non,Non,Oui,Non,300.000000,25.00,user748aa2@example.com,YF632342
1,46,Senior,Femme,Urbain,Centre,Célibataire,Supérieur,14.0,Cadres supérieurs,Public,Oui,Oui,Non,Non,78506.710725,6542.23,user861b7d@example.com,DU412942
2,32,Adulte,Homme,Rural,Sud,Marié,Fondamental,8.0,Ouvriers,Informel,Non,Oui,Non,Non,395.248250,32.94,user0e21f6@example.com,UQ738551
3,60,Âgé,Homme,Rural,Sud,Marié,Supérieur,17.0,Cadres supérieurs,Privé,Oui,Oui,Non,Oui,105781.533607,8815.13,user91e502@example.com,GE492077
4,25,Adulte,Femme,Urbain,Centre,Marié,Secondaire,3.0,Employés,Privé,Non,Non,Non,Non,22016.268128,1834.69,userdb161f@example.com,YF665579
5,38,Adulte,Homme,Rural,Est,Divorcé,Secondaire,5.0,Employés,Privé,Oui,Non,Non,Non,1995.320311,166.28,userf12e9f@example.com,YQ100599
6,56,Senior,Homme,Rural,Ouest,Marié,Fondamental,33.0,Ouvriers,Privé,Non,Non,Non,Non,1176.015258,98.00,user782a30@example.com,TK612340
7,36,Adulte,Femme,Urbain,Ouest,Marié,Secondaire,7.0,Employés,Privé,Oui,Oui,Non,Non,22195.154959,1849.60,user0c5223@example.com,AD480612
8,40,Adulte,Homme,Urbain,Centre,Marié,Supérieur,13.0,Cadres supérieurs,Privé,Oui,Non,Non,Non,92564.488887,7713.71,user241e19@example.com,ZJ351083
9,28,Adulte,Femme,Urbain,Nord,Marié,Secondaire,3.0,Ouvriers,Privé,Non,Non,Non,Non,17172.720964,1431.06,user6bfb9f@example.com,BH694916



Aperçu des 10 dernières lignes du DataFrame :


,Age,Categorie_age,Sexe,Milieu,Region_geographique,Etat_matrimonial,Niveau_education,Annees_experience,CSP,Secteur_emploi,Propriete_immobiliere,Vehicule_motorise,Terrain_agricole,Revenu_secondaire,Revenu_Annuel,Revenu_Mensuel,Adresse_Email,CIN
39990,26,Adulte,Homme,Urbain,Nord,Veuf,Secondaire,5.0,Employés,Public,Non,Non,Non,Oui,22982.650695,1915.22,userf32442@example.com,YJ411569
39991,40,Adulte,Homme,Rural,Nord,Célibataire,Secondaire,1.0,Ouvriers,Privé,Non,Non,Non,Non,1123.483370,93.62,user8e8e2c@example.com,PN139847
39992,24,Jeune,Femme,Urbain,Sud,Célibataire,Fondamental,1.0,Agriculteurs,Informel,Non,Non,Oui,Oui,11724.909786,977.08,usereef9f8@example.com,UP616808
39993,48,Senior,Femme,Urbain,Centre,Marié,Secondaire,25.0,Professions intermédiaires,Public,Non,Non,Oui,Non,38373.940680,3197.83,userbe625c@example.com,TQ978793
39994,38,Adulte,Femme,Urbain,Nord,Célibataire,Supérieur,13.0,Cadres supérieurs,Privé,Oui,Oui,Non,Oui,72210.412302,6017.53,userd32e41@example.com,SQ690182
39995,18,Jeune,Femme,Urbain,Centre,Marié,Sans niveau,0.0,Inactifs,NaN,Non,Non,Non,Non,2652.218311,221.02,useref70b8@example.com,MV264884
39996,63,Âgé,Homme,Urbain,Ouest,Divorcé,Supérieur,30.0,Cadres supérieurs,Public,Non,Oui,Non,Oui,146859.531881,12238.29,user5d6c5c@example.com,QN622528
39997,19,Jeune,Femme,Rural,Sud,Célibataire,Fondamental,0.0,Inactifs,NaN,Non,Non,Non,Non,300.000000,25.00,userea297d@example.com,JG342827
39998,57,Senior,Homme,Urbain,Nord,Marié,Secondaire,21.0,Professions intermédiaires,Privé,Non,Non,Non,Oui,47636.743845,3969.73,useref6bca@example.com,GW748819
39999,28,Adulte,Homme,Urbain,Nord,Marié,Sans niveau,1.0,Inactifs,NaN,Oui,Non,Non,Oui,3926.534904,327.21,user87b9ed@example.com,UR878598


## 1.3 Description Générale des Données (Types et Valeurs Manquantes)
Pour obtenir une vue synthétique des types de données de chaque colonne et du nombre de valeurs non nulles (ce qui nous indique les valeurs manquantes), nous utilisons la méthode `info()` du DataFrame.

In [111]:
# --- VÉRIFICATION DU CHARGEMENT AVANT D'AFFICHER LES INFORMATIONS ---
# On vérifie à nouveau si 'df' contient des données avant de demander ses caractéristiques.
if df is not None:

    # --- 1. OBTENTION DES INFORMATIONS GÉNÉRALES SUR LE DATAFRAME ---
    # On utilise une fonction très pratique de pandas pour avoir un résumé technique du DataFrame.
    print("Informations générales sur le DataFrame (types, nombre de valeurs non nulles par colonne) :")
    df.info()
    # df.info() nous donne :
    # - Le nombre total d'entrées (lignes).
    # - Le nombre de colonnes.
    # - Pour chaque colonne : son nom, le nombre de valeurs non manquantes (non-null), et son type de données (Dtype).
    # - La quantité de mémoire utilisée par le DataFrame.
else:
    # --- GESTION DU CAS OÙ LES DONNÉES NE SONT PAS CHARGÉES ---
    # Si 'df' est vide :
    print("ATTENTION : Le DataFrame 'df' n'est pas chargé. Impossible d'afficher les informations.")

Informations générales sur le DataFrame (types, nombre de valeurs non nulles par colonne) :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Age                    40000 non-null  int64  
 1   Categorie_age          39960 non-null  object 
 2   Sexe                   40000 non-null  object 
 3   Milieu                 40000 non-null  object 
 4   Region_geographique    40000 non-null  object 
 5   Etat_matrimonial       39965 non-null  object 
 6   Niveau_education       40000 non-null  object 
 7   Annees_experience      39960 non-null  float64
 8   CSP                    40000 non-null  object 
 9   Secteur_emploi         37390 non-null  object 
 10  Propriete_immobiliere  39958 non-null  object 
 11  Vehicule_motorise      39965 non-null  object 
 12  Terrain_agricole       39969 non-null  object 
 13  Revenu_seconda

## 1.4 Statistiques Descriptives
Nous allons maintenant calculer les statistiques descriptives (moyenne, écart-type, min, max, quartiles, etc.) pour les variables numériques, et les statistiques de fréquence (nombre d'uniques, valeur la plus fréquente) pour les variables catégorielles.

In [112]:
# --- VÉRIFICATION DU CHARGEMENT AVANT DE CALCULER LES STATISTIQUES ---
# On s'assure que 'df' contient des données avant de faire des calculs.
if df is not None:

    # --- 1. STATISTIQUES DESCRIPTIVES POUR LES COLONNES NUMÉRIQUES ---
    # On calcule des mesures statistiques clés pour les colonnes numériques.
    print("Statistiques descriptives pour les colonnes numériques :")
    numeric_stats = (df.select_dtypes(include=np.number)  # Étape 1: Sélectionne les colonnes numériques.
                       .describe()                        # Étape 2: Calcule les statistiques.
                       .T)                                # Étape 3: Transpose pour meilleure lisibilité.
    display(numeric_stats)

    # --- 2. STATISTIQUES DESCRIPTIVES POUR LES COLONNES CATÉGORIELLES ---
    # On regarde les statistiques adaptées aux colonnes de texte ou catégories.
    print("\nStatistiques descriptives pour les colonnes catégorielles (type 'object' et 'category') :")
    categorical_stats = (df.select_dtypes(include=['object', 'category']) # Étape 1: Sélectionne les colonnes texte/catégorie.
                           .describe()                                   # Étape 2: Calcule les statistiques adaptées.
                           .T)                                            # Étape 3: Transpose.
    display(categorical_stats)

    # --- 3. NOMBRE DE VALEURS UNIQUES PAR COLONNE ---
    # On compte combien de valeurs différentes il y a dans chaque colonne.
    print("\nNombre de valeurs uniques par colonne (trié par ordre décroissant) :")
    unique_counts = (df.nunique()                             # Étape 1: Calcule le nombre de valeurs uniques.
                       .sort_values(ascending=False)         # Étape 2: Trie les résultats.
                       .to_frame(name='Nombre de Valeurs Uniques')) # Étape 3: Transforme en DataFrame pour affichage.
    display(unique_counts)
else:
    # --- GESTION DU CAS OÙ LES DONNÉES NE SONT PAS CHARGÉES ---
    # Si 'df' est vide :
    print("ATTENTION : Le DataFrame 'df' n'est pas chargé. Impossible de calculer les statistiques.")

Statistiques descriptives pour les colonnes numériques :


,count,mean,std,min,25%,50%,75%,max
Age,40000.0,40.512625,13.509623,-5.00,29.000000,40.000000,52.000000,150.00
Annees_experience,39960.0,10.401051,9.890290,0.00,2.000000,8.000000,16.000000,45.00
Revenu_Annuel,40000.0,21505.790212,33809.163803,200.00,2211.446624,10500.621766,26107.364841,700000.00
Revenu_Mensuel,40000.0,1792.149185,2817.430317,16.67,184.287500,875.050000,2175.612500,58333.33



Statistiques descriptives pour les colonnes catégorielles (type 'object' et 'category') :


,count,unique,top,freq
Categorie_age,39960,4,Adulte,17615
Sexe,40000,2,Homme,20817
Milieu,40000,2,Urbain,23937
Region_geographique,40000,5,Centre,11212
Etat_matrimonial,39965,4,Marié,21297
Niveau_education,40000,4,Secondaire,13974
CSP,40000,6,Ouvriers,14975
Secteur_emploi,37390,3,Privé,17624
Propriete_immobiliere,39958,2,Non,29202
Vehicule_motorise,39965,2,Non,28477



Nombre de valeurs uniques par colonne (trié par ordre décroissant) :


,Nombre de Valeurs Uniques
CIN,39999
Adresse_Email,39956
Revenu_Annuel,34300
Revenu_Mensuel,32396
Age,48
Annees_experience,46
CSP,6
Region_geographique,5
Etat_matrimonial,4
Niveau_education,4


## 1.5 Exploration Visuelle Approfondie (EDA avec ydata-profiling)
Pour obtenir une compréhension encore plus profonde et visuelle des données, notamment les distributions de chaque variable, les corrélations, les interactions et les valeurs manquantes, nous allons utiliser la librairie `ydata-profiling` (anciennement Pandas Profiling). Elle génère un rapport HTML interactif qui fournit une analyse détaillée de chaque variable du dataset.


In [ ]:
# --- VÉRIFICATION PRÉALABLE DE L'EXISTENCE ET DU CHARGEMENT DU DATAFRAME ---
# On s'assure que la variable 'df' existe et qu'elle contient des données avant d'essayer de l'utiliser.
if 'df' in locals() and df is not None:
  
    # --- TENTATIVE D'UTILISATION DE YDATA-PROFILING ---
    # On va essayer d'utiliser la librairie ydata-profiling. Comme elle n'est pas toujours installée par défaut,
    try:
        # --- 1. IMPORTATION DE L'OUTIL DE PROFILING ---
        # On importe la classe ProfileReport de la librairie ydata_profiling.
        from ydata_profiling import ProfileReport

        print("Génération du rapport avec ydata-profiling... Cela peut prendre quelques instants.")

        # --- 2. CRÉATION DU RAPPORT ---
        # On crée un objet "ProfileReport" en lui donnant notre DataFrame 'df'.
        # On peut aussi personnaliser le titre du rapport et activer des options exploratoires.
        profile = ProfileReport(df,
                                title="Rapport d'Exploration Détaillée - Revenu des Marocains",
                                explorative=True) # Active des analyses plus poussées (corrélations, etc.).

        # --- 3. SAUVEGARDE DU RAPPORT EN FICHIER HTML ---
        # On définit un nom pour le fichier HTML qui sera généré.
        profile_filename = "Pandas_Profiling_Report_Revenu_Marocains.html"
        # On sauvegarde le rapport dans ce fichier.
        profile.to_file(profile_filename)

        # --- 4. CONFIRMATION ET INSTRUCTIONS ---
        # On informe l'utilisateur que le rapport a été généré et où il se trouve.
        print(f"\nRapport ydata-profiling généré et sauvegardé sous : '{profile_filename}'")
        print(f"Veuillez ouvrir le fichier '{profile_filename}' manuellement depuis votre explorateur de fichiers pour le consulter.")

    except ImportError:
        # --- GESTION DE L'ERREUR SI YDATA-PROFILING N'EST PAS INSTALLÉ ---
        print("ERREUR d'importation: La librairie ydata-profiling n'est pas installée.")
        print("Veuillez l'installer (par exemple, avec : pip install ydata-profiling) dans l'environnement Python utilisé par VS Code.")
    except Exception as e:
        # --- GESTION DES AUTRES ERREURS POSSIBLES ---
        print(f"Une erreur générale est survenue lors de la génération du rapport ydata-profiling : {e}")
else:
    # --- MESSAGE SI LE DATAFRAME N'EST PAS PRÊT ---
    print("ATTENTION : Le DataFrame 'df' n'est pas chargé ou n'est pas défini. Veuillez exécuter les cellules précédentes pour charger 'df'.")

Génération du rapport avec ydata-profiling... Cela peut prendre quelques instants.


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:06<00:00,  2.90it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]


Rapport ydata-profiling généré et sauvegardé sous : 'Pandas_Profiling_Report_Revenu_Marocains.html'
Veuillez ouvrir le fichier 'Pandas_Profiling_Report_Revenu_Marocains.html' manuellement depuis votre explorateur de fichiers pour le consulter.


## 1.6 Bilan de la Compréhension des Données

Après avoir examiné les statistiques descriptives (`df.info()`, `df.describe()`) et le rapport d'exploration détaillé généré par `ydata-profiling` (version v4.16.1), nous pouvons synthétiser nos principales observations concernant le dataset :

*   **Structure Générale :**
    *   Le dataset contient **40 000 observations** et **18 variables**.
    *   Il a été confirmé par le rapport qu'il n'y a **aucune ligne dupliquée** (`Duplicate rows: 0`).
    *   Le nombre total de cellules manquantes est de **2871**, ce qui représente **0.4%** de l'ensemble des données (`Missing cells: 2871`, `Missing cells (%): 0.4%`).

*   **Variable Cible (`Revenu_Annuel`) :**
    *   Elle présente une distribution fortement asymétrique à droite (Skewness = **4.53**), avec une Kurtosis élevée (**39.18**), indiquant des queues épaisses (plus d'outliers). La moyenne (Mean) est d'environ **21 506 DH**, tandis que la médiane est significativement plus basse, à environ **10 501 DH**. Cette asymétrie est typique des données de revenu.
    *   Les valeurs s'étendent de **200 DH** (Minimum) à **700 000 DH** (Maximum).
    *   Aucune valeur manquante n'est présente pour cette variable cible (`Missing: 0`).

*   **Variables Explicatives Numériques :**
    *   `Age` : Les valeurs varient de **-5** (Minimum) à **150** ans (Maximum), confirmant la présence d'aberrations introduites (14 valeurs négatives signalées). Le rapport signale une forte corrélation avec `Annees_experience` (coefficient de **0.682**) et `Categorie_age` (coefficient de **0.815**). Aucune valeur manquante.
    *   `Annees_experience` : Comporte **40 valeurs manquantes** (0.1%). Un nombre notable de **5251 observations (13.1%** de la colonne) présentent une valeur de zéro (alert: `Annees_experience has 5251 (13.1%) zeros`), indiquant des individus sans expérience professionnelle. Elle est également fortement corrélée à `Age`.

*   **Variables Explicatives Catégorielles et Textuelles :**
    *   `Categorie_age` : Comporte **40 valeurs manquantes** (0.1%), probablement dues aux valeurs aberrantes de `Age`. Elle est logiquement très corrélée à `Age`. La catégorie la plus fréquente est "Adulte" (17615 occurrences).
    *   `Sexe`, `Milieu`, `Region_geographique`, `Niveau_education`, `CSP` : Ne présentent pas de valeurs manquantes. Leurs répartitions (ex: "Homme" 20817 pour `Sexe`, "Urbain" 23937 pour `Milieu`, "Secondaire" 13974 pour `Niveau_education`, "Ouvriers" 14975 pour `CSP`) sont conformes à la logique de génération. `Niveau_education` et `CSP` sont signalées comme fortement corrélées.
    *   `Etat_matrimonial` : Présente **35 valeurs manquantes** (0.1%). "Marié" est la catégorie la plus fréquente (21297).
    *   `Secteur_emploi` : A le plus grand nombre de valeurs manquantes avec **2610 occurrences (6.5%** des données pour cette colonne, alert: `Secteur_emploi has 2610 (6.5%) missing values`), ce qui est cohérent avec les "Inactifs" qui n'ont pas de secteur d'emploi. "Privé" est la catégorie la plus fréquente (17624).
    *   `Propriete_immobiliere` (**42 NaN**, 0.1%), `Vehicule_motorise` (**35 NaN**, 0.1%), `Terrain_agricole` (**31 NaN**, 0.1%), `Revenu_secondaire` (**38 NaN**, 0.1%) : Toutes ces variables binaires présentent un faible pourcentage de valeurs manquantes.
    *   `Terrain_agricole` est signalé par `ydata-profiling` comme étant déséquilibré (alert: `Terrain_agricole is highly imbalanced (56.0%)` - *Note: la valeur de 56.0% pour l'alerte d'imbalance est une interprétation de ydata-profiling; les données brutes indiquent environ 90.9% de 'Non'. L'alerte d'imbalance reste pertinente.*). Il est aussi fortement corrélé avec `CSP`.

*   **Colonnes Identifiants (Type Texte) :**
    *   `Adresse_Email` (**39956 valeurs distinctes** sur 40000) et `CIN` (**39999 valeurs distinctes** sur 40000) : Possèdent une très haute cardinalité (presque uniques pour chaque enregistrement), confirmant leur nature d'identifiants.

*   **Colonnes à considérer pour suppression (Redondance/Non Pertinence) :**
    *   `Revenu_Mensuel` : Identifié comme parfaitement corrélé avec `Revenu_Annuel` (coefficient de **1.000**). Sa suppression sera effectuée.
    *   `Adresse_Email` et `CIN` : Seront supprimées en raison de leur haute cardinalité et non-pertinence pour la prédiction.
    *   `Categorie_age` : Étant directement dérivée de `Age` et fortement corrélée (**0.815**), sa pertinence sera réévaluée. Si `Age` est conservé, `Categorie_age` pourrait être redondante.

*   **Corrélations Notables (basées sur la table de corrélation de Pearson du rapport) :**
    *   Variables les plus positivement corrélées avec `Revenu_Annuel` :
        *   `CSP` (coeff. **0.390**)
        *   `Niveau_education` (coeff. **0.305**)
        *   `Propriete_immobiliere` (coeff. **0.263**)
        *   `Vehicule_motorise` (coeff. **0.248**)
        *   `Annees_experience` (coeff. **0.196**)
        *   `Age` (coeff. **0.178**)
    *   Le rapport HTML complet de `ydata-profiling` devrait être consulté pour des métriques de corrélation plus adaptées aux variables mixtes (ex: Phik, Cramér's V).
    *   **Multicollinéarité** significative entre les variables explicatives (confirmée par les alertes de `ydata-profiling`) :
        *   `Age` et `Annees_experience` (**0.682**)
        *   `CSP` et `Niveau_education` (**0.640**)
        *   `CSP` et `Terrain_agricole` (**0.714**)
        *   `CSP` et `Secteur_emploi` (**0.473**)
        *   `CSP` et `Propriete_immobiliere` (**0.454**)
        *   `CSP` et `Vehicule_motorise` (**0.424**)
        *   `Niveau_education` et `Secteur_emploi` (**0.396**)
        *   `Niveau_education` et `Vehicule_motorise` (**0.335**)
        La gestion de cette multicollinéarité sera importante.

*   **Alertes du rapport `ydata-profiling` (Synthèse des 11 alertes mentionnées) :**
    1.  `Age` fortement corrélé avec `Annees_experience` et `Categorie_age`.
    2.  `Annees_experience` fortement corrélé avec `Age`.
    3.  `CSP` fortement corrélé avec `Niveau_education` et `Terrain_agricole`.
    4.  `Categorie_age` fortement corrélé avec `Age`.
    5.  `Niveau_education` fortement corrélé avec `CSP`.
    6.  `Revenu_Annuel` fortement corrélé avec `Revenu_Mensuel`.
    7.  `Revenu_Mensuel` fortement corrélé avec `Revenu_Annuel`.
    8.  `Terrain_agricole` fortement corrélé avec `CSP`.
    9.  `Terrain_agricole` est déséquilibré.
    10. `Secteur_emploi` a 6.5% de valeurs manquantes.
    11. `Annees_experience` a 13.1% de zéros.

**Conclusion de l'EDA et Prochaines Étapes :**
L'exploration des données nous a fourni une compréhension approfondie du dataset, de ses caractéristiques individuelles, de la qualité des données (présence de NaN, d'outliers intentionnels, et de zéros) et des relations et corrélations potentielles entre les variables. Les données générées semblent bien refléter les intentions initiales, y compris les imperfections et les relations attendues.
Les prochaines étapes se concentreront sur la préparation de ces données pour la modélisation :
1.  **Séparation des données** en ensembles d'apprentissage et de test.
2.  **Nettoyage des données**, ce qui inclura le traitement des valeurs manquantes et des valeurs aberrantes identifiées. (Les lignes dupliquées ne sont pas un problème ici).
3.  **Transformation des données**, comprenant la suppression des colonnes jugées non pertinentes ou redondantes, l'encodage approprié des variables catégorielles en format numérique, et potentiellement la normalisation ou la standardisation des variables numériques. La gestion de la multicollinéarité sera également considérée.
Ces opérations seront, autant que possible, intégrées dans des pipelines `scikit-learn` pour assurer un processus de prétraitement structuré, reproductible et applicable de manière cohérente aux différents sous-ensembles de données.

# 2. Préparation des Données

Après avoir exploré et compris notre dataset, cette section est dédiée à sa préparation en vue de la modélisation. Cela implique plusieurs étapes cruciales :
*   La suppression des colonnes jugées inutiles ou redondantes.
*   La séparation des données en ensembles d'apprentissage (`X_train`, `y_train`) et de test (`X_test`, `y_test`).
*   Le nettoyage des données, notamment le traitement des valeurs manquantes et des valeurs aberrantes.
*   La transformation des données, incluant l'encodage des variables catégorielles en format numérique et la normalisation/standardisation des variables numériques.

Conformément au cahier des charges, ces étapes de nettoyage et de transformation seront, autant que possible, intégrées dans des **pipelines `scikit-learn`** pour un traitement structuré et reproductible.

## 2.1 Suppression des Colonnes Inutiles et Définition des Features (X) et de la Cible (y)

Basé sur notre analyse exploratoire (section 1.6), nous allons d'abord supprimer les colonnes identifiées comme non pertinentes (identifiants uniques) ou redondantes. Ensuite, nous séparerons notre DataFrame en un ensemble de variables explicatives (features `X`) et la variable cible (`y`).

In [113]:
# --- VÉRIFICATION PRÉALABLE DE L'EXISTENCE ET DU CHARGEMENT DU DATAFRAME ---
if 'df' in locals() and df is not None:

    # --- 1. COPIE DU DATAFRAME POUR LE TRAITEMENT ---
    # On crée une copie de 'df' pour ne pas modifier l'original. C'est une bonne pratique.
    df_processed = df.copy()

    # --- 2. DÉFINITION ET SÉPARATION DE LA VARIABLE CIBLE (y) ---
    # La variable cible est celle que l'on chercherait à prédire dans un modèle.
    TARGET_COL = 'Revenu_Annuel' 
    y = df_processed[TARGET_COL] # On extrait cette colonne pour la stocker dans 'y'.

    # --- 3. DÉFINITION DES COLONNES À SUPPRIMER POUR LES FEATURES (X) ---
    cols_to_drop_initial = ['Revenu_Mensuel', 'Adresse_Email', 'CIN', 'Categorie_age']

    # Pour éviter une erreur si une colonne listée n'existe pas déjà dans le DataFrame :
    # On ne garde dans notre liste que les colonnes qui sont réellement présentes dans df_processed.
    actual_cols_to_drop = [col for col in cols_to_drop_initial if col in df_processed.columns]

    # --- 4. CRÉATION DU DATAFRAME DES FEATURES (X) ---
    # 'X' contiendra toutes les colonnes SAUF la colonne cible ET les colonnes qu'on a décidé de supprimer.
    X = df_processed.drop(columns=[TARGET_COL] + actual_cols_to_drop, errors='ignore')

    # --- 5. AFFICHAGE D'INFORMATIONS SUR X ET y ---
    print("Colonnes initialement supprimées de X (en plus de la cible) :", actual_cols_to_drop)
    print(f"Nombre de colonnes restantes dans X avant séparation train/test : {X.shape[1]}") # X.shape[1] donne le nombre de colonnes
    print("Liste des colonnes dans X :")
    print(X.columns.tolist()) # Affiche la liste des noms des colonnes de X

    print(f"\nDimensions de X : {X.shape}")
    print(f"Dimensions de y : {y.shape}") 

    print("\nAperçu de X (5 premières lignes) :")
    display(X.head()) 
    print("\nAperçu de y (5 premières valeurs) :")
    display(y.head()) 

else:
    # --- MESSAGE SI LE DATAFRAME N'EST PAS PRÊT ---
    print("ATTENTION : Le DataFrame 'df' n'est pas chargé ou n'est pas défini.")

Colonnes initialement supprimées de X (en plus de la cible) : ['Revenu_Mensuel', 'Adresse_Email', 'CIN', 'Categorie_age']
Nombre de colonnes restantes dans X avant séparation train/test : 13
Liste des colonnes dans X :
['Age', 'Sexe', 'Milieu', 'Region_geographique', 'Etat_matrimonial', 'Niveau_education', 'Annees_experience', 'CSP', 'Secteur_emploi', 'Propriete_immobiliere', 'Vehicule_motorise', 'Terrain_agricole', 'Revenu_secondaire']

Dimensions de X : (40000, 13)
Dimensions de y : (40000,)

Aperçu de X (5 premières lignes) :


,Age,Sexe,Milieu,Region_geographique,Etat_matrimonial,Niveau_education,Annees_experience,CSP,Secteur_emploi,Propriete_immobiliere,Vehicule_motorise,Terrain_agricole,Revenu_secondaire
0,56,Homme,Rural,Centre,Marié,Fondamental,5.0,Agriculteurs,Privé,Non,Non,Oui,Non
1,46,Femme,Urbain,Centre,Célibataire,Supérieur,14.0,Cadres supérieurs,Public,Oui,Oui,Non,Non
2,32,Homme,Rural,Sud,Marié,Fondamental,8.0,Ouvriers,Informel,Non,Oui,Non,Non
3,60,Homme,Rural,Sud,Marié,Supérieur,17.0,Cadres supérieurs,Privé,Oui,Oui,Non,Oui
4,25,Femme,Urbain,Centre,Marié,Secondaire,3.0,Employés,Privé,Non,Non,Non,Non



Aperçu de y (5 premières valeurs) :


0       300.000000
1     78506.710725
2       395.248250
3    105781.533607
4     22016.268128
Name: Revenu_Annuel, dtype: float64

## 2.2 Séparation des Données en Ensembles d'Apprentissage et de Test

Avant de procéder au nettoyage et à la transformation plus poussée des données, il est crucial de diviser notre dataset (`X` et `y`) en deux sous-ensembles distincts :
*   Un **ensemble d'apprentissage** (`X_train`, `y_train`) : Ce sous-ensemble, représentant 70% des données, sera utilisé pour entraîner nos modèles de Machine Learning. C'est sur ces données que les modèles "apprendront" les relations entre les caractéristiques et la variable cible.
*   Un **ensemble de test** (`X_test`, `y_test`) : Ce sous-ensemble, représentant les 30% restants, sera mis de côté et utilisé uniquement à la fin du processus pour évaluer la performance des modèles entraînés sur des données qu'ils n'ont jamais vues auparavant. Cela permet d'obtenir une estimation impartiale de leur capacité à généraliser.

Cette séparation est fondamentale pour éviter le surapprentissage (overfitting), où un modèle apprendrait trop spécifiquement les données d'entraînement (y compris leur bruit) et performerait mal sur de nouvelles données. Nous utilisons la fonction `train_test_split` de `scikit-learn` pour cette opération, en spécifiant un `random_state` pour garantir la reproductibilité de la division.

In [114]:
# --- IMPORTATION DE L'OUTIL DE DIVISION DES DONNÉES ---
# On importe la fonction 'train_test_split' de la librairie scikit-learn (sklearn)
from sklearn.model_selection import train_test_split

# --- VÉRIFICATION DE L'EXISTENCE DE X ET y ---
if 'X' in locals() and 'y' in locals():

    # --- 1. DIVISION DES DONNÉES EN ENSEMBLES D'ENTRAÎNEMENT ET DE TEST ---
    # On utilise train_test_split pour séparer nos données X et y en quatre parties :
    X_train, X_test, y_train, y_test = train_test_split(
        X,                          # Notre DataFrame de features
        y,                          # Notre Série de la variable cible
        test_size=0.30,             # Proportion des données à mettre dans l'ensemble de test (ici 30%).
        random_state=RANDOM_SEED    # On utilise la même graine aléatoire (RANDOM_SEED définie au début)
                                    # pour que la division soit toujours la même si on relance le code.
    )

    # --- 2. AFFICHAGE DES DIMENSIONS DES NOUVEAUX ENSEMBLES ---
    # On vérifie les tailles (nombre de lignes, nombre de colonnes) des DataFrames et Séries créés.
    print("Dimensions des ensembles de données après séparation :")
    print(f"  X_train: {X_train.shape}") 
    print(f"  y_train: {y_train.shape}")
    print(f"  X_test: {X_test.shape}")  
    print(f"  y_test: {y_test.shape}")

    # --- 3. AFFICHAGE DES PROPORTIONS ---
    print("\nProportion des données dans chaque ensemble :")
    print(f"  Apprentissage (train) : {len(X_train) / len(X):.0%}") 
    print(f"  Test : {len(X_test) / len(X):.0%}")                 

    # --- 4. APERÇU DE X_train ---
    print("\nAperçu de X_train (5 premières lignes) :")
    display(X_train.head())

else:
    # --- MESSAGE SI X OU y NE SONT PAS PRÊTS ---
    print("ATTENTION : Les variables X et/ou y ne sont pas définies. Veuillez exécuter la cellule précédente.")

Dimensions des ensembles de données après séparation :
  X_train: (28000, 13)
  y_train: (28000,)
  X_test: (12000, 13)
  y_test: (12000,)

Proportion des données dans chaque ensemble :
  Apprentissage (train) : 70%
  Test : 30%

Aperçu de X_train (5 premières lignes) :


,Age,Sexe,Milieu,Region_geographique,Etat_matrimonial,Niveau_education,Annees_experience,CSP,Secteur_emploi,Propriete_immobiliere,Vehicule_motorise,Terrain_agricole,Revenu_secondaire
38015,34,Femme,Urbain,Sud,Célibataire,Secondaire,2.0,Employés,Privé,Oui,Non,Non,Non
2281,21,Homme,Rural,Est,Célibataire,Fondamental,2.0,Agriculteurs,Informel,Oui,Oui,Oui,Non
36629,56,Homme,Rural,Ouest,Célibataire,Fondamental,19.0,Ouvriers,Informel,Non,Non,Non,Non
6087,25,Femme,Urbain,Nord,Divorcé,Secondaire,2.0,Ouvriers,Privé,Non,Non,Non,Non
11792,35,Homme,Urbain,Ouest,Marié,Secondaire,7.0,Employés,Privé,Non,Non,Non,Non


## 2.3 Identification des Types de Colonnes pour les Pipelines de Prétraitement

Avant de construire nos pipelines, nous devons identifier clairement quelles colonnes de notre ensemble d'apprentissage `X_train` sont numériques et lesquelles sont catégorielles. Ces deux types de colonnes nécessiteront des traitements différents (par exemple, imputation et normalisation pour les numériques, imputation et encodage pour les catégorielles).

Nous identifierons également la colonne `Age` séparément car elle contient des valeurs aberrantes spécifiques (-5, 150) que nous pourrions vouloir traiter d'une manière particulière. De même pour `Annees_experience` qui contient des zéros et des valeurs manquantes.

In [115]:
# --- VÉRIFICATION DE L'EXISTENCE DE X_train ---
if 'X_train' in locals() and X_train is not None:

    # --- 1. IDENTIFICATION DES COLONNES NUMÉRIQUES ---
    # On sélectionne toutes les colonnes de X_train qui contiennent des données de type numérique (nombres entiers, décimaux).
    numerical_cols = X_train.select_dtypes(include=np.number).columns.tolist()

    print("Colonnes Numériques Identifiées :")
    print(numerical_cols) # Affiche la liste des colonnes numériques.
    print(f"Nombre de colonnes numériques : {len(numerical_cols)}") # Affiche combien il y en a.

    # --- 2. IDENTIFICATION DES COLONNES CATÉGORIELLES ---
    # On sélectionne toutes les colonnes de X_train qui contiennent des données de type 'object' (souvent du texte) ou 'category'.
    categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()


    print("\nColonnes Catégorielles Identifiées :")
    print(categorical_cols) # Affiche la liste des colonnes catégorielles.
    print(f"Nombre de colonnes catégorielles : {len(categorical_cols)}") # Affiche combien il y en a.

    # --- 3. VÉRIFICATION DE LA COHÉRENCE DU NOMBRE DE COLONNES ---
    if len(numerical_cols) + len(categorical_cols) == X_train.shape[1]:
        # X_train.shape[1] donne le nombre total de colonnes dans X_train.
        print("\nLa somme des colonnes numériques et catégorielles correspond bien au nombre total de colonnes dans X_train.")
    else:
        print("\nATTENTION : Incohérence dans le nombre de colonnes. Vérifiez les types.")
        print(f"Total colonnes X_train: {X_train.shape[1]}")
        print(f"Numériques + Catégorielles: {len(numerical_cols) + len(categorical_cols)}")

else:
    # --- MESSAGE SI X_train N'EST PAS PRÊT ---
    print("ATTENTION : X_train n'est pas défini. Veuillez exécuter les cellules précédentes.")

Colonnes Numériques Identifiées :
['Age', 'Annees_experience']
Nombre de colonnes numériques : 2

Colonnes Catégorielles Identifiées :
['Sexe', 'Milieu', 'Region_geographique', 'Etat_matrimonial', 'Niveau_education', 'CSP', 'Secteur_emploi', 'Propriete_immobiliere', 'Vehicule_motorise', 'Terrain_agricole', 'Revenu_secondaire']
Nombre de colonnes catégorielles : 11

La somme des colonnes numériques et catégorielles correspond bien au nombre total de colonnes dans X_train.


## 2.4 Création des Pipelines de Prétraitement

Nous allons maintenant définir des pipelines distincts pour les variables numériques et catégorielles en utilisant `make_pipeline` et `ColumnTransformer` de Scikit-learn.

**Pour les variables numériques, le pipeline inclura :**
1.  **Traitement des valeurs manquantes (`SimpleImputer`) :** Nous utiliserons la médiane pour imputer les NaN, car elle est moins sensible aux outliers que la moyenne.
2.  **Traitement des valeurs aberrantes :** Pour la colonne `Age`, nous appliquerons une technique de "clipping" (plafonnement) pour ramener les valeurs aberrantes dans une plage plus raisonnable (par exemple, 18 à 70 ans). Pour les autres colonnes numériques, nous pourrions envisager `RobustScaler` qui est moins sensible aux outliers, ou d'autres techniques si nécessaire.
3.  **Normalisation/Standardisation (`StandardScaler` ou `RobustScaler`) :** Pour mettre toutes les variables numériques sur une échelle comparable, ce qui est important pour de nombreux algorithmes (Régression Linéaire, MLP, etc.). `StandardScaler` centre les données autour de 0 avec un écart-type de 1. `RobustScaler` utilise les quantiles et est plus robuste aux outliers.

**Pour les variables catégorielles, le pipeline inclura :**
1.  **Traitement des valeurs manquantes (`SimpleImputer`) :** Nous remplacerons les NaN par la valeur la plus fréquente de la colonne ou par une constante comme "Manquant".
2.  **Encodage (`OneHotEncoder`) :** Nous transformerons les catégories en un ensemble de colonnes binaires. Nous utiliserons `handle_unknown='ignore'` pour gérer les catégories rares qui pourraient apparaître dans l'ensemble de test mais pas dans l'ensemble d'apprentissage.

Ces pipelines individuels seront ensuite combinés à l'aide d'un `ColumnTransformer`.

In [117]:
# --- 1. IMPORTATION DES OUTILS NÉCESSAIRES POUR LE PRÉTRAITEMENT ---
# On importe divers outils de scikit-learn pour construire nos chaînes de transformation de données.
from sklearn.pipeline import Pipeline # Pour créer des séquences d'opérations (ex: imputer PUIS mettre à l'échelle).
from sklearn.impute import SimpleImputer # Pour remplacer les valeurs manquantes (NaN).
from sklearn.preprocessing import StandardScaler, OneHotEncoder # StandardScaler pour normaliser les données numériques ; OneHotEncoder pour transformer les catégories en nombres.
from sklearn.compose import ColumnTransformer # Pour appliquer différents pipelines à différentes colonnes.
# Les deux lignes suivantes sont généralement pour créer des transformateurs personnalisés,
from sklearn.base import BaseEstimator, TransformerMixin


# La classe AgeClipper
# Elle servirait à limiter les valeurs d'une colonne 'Age' entre un min et un max.
class AgeClipper(BaseEstimator, TransformerMixin):
    def __init__(self, min_age=18, max_age=75): 
        self.min_age = min_age
        self.max_age = max_age
    def fit(self, X, y=None): 
        return self
    def transform(self, X, y=None): # La méthode transform applique le clipping.
        X_transformed_np = np.copy(X) 
        X_transformed_np = np.clip(X_transformed_np, self.min_age, self.max_age)
        return X_transformed_np

# --- 2. VÉRIFICATION DE LA DISPONIBILITÉ DES DONNÉES NÉCESSAIRES ---
# On s'assure que X_train et les listes de colonnes numériques/catégorielles (identifiées précédemment) existent.
if ('X_train' in locals() and
    'numerical_cols' in locals() and
    'categorical_cols' in locals()):

    # --- 3. DÉFINITION DES PIPELINES DE PRÉTRAITEMENT SPÉCIFIQUES ---

    # --- 3.a. Pipeline pour la colonne 'Age' (si elle existe) ---
    # Séquence de traitements pour la colonne 'Age'.
    age_pipeline = Pipeline([
        ('imputer_age', SimpleImputer(strategy='median')), # Étape 1: Remplace les valeurs manquantes (NaN) dans 'Age' par la médiane des âges.
        ('scaler_age', StandardScaler())                   # Étape 2: Normalise les valeurs de 'Age' (centre autour de 0, écart-type de 1).
    ])

    # --- 3.b. Pipeline pour la colonne 'Annees_experience' (si elle existe) ---
    # Séquence de traitements pour la colonne 'Annees_experience'.
    experience_pipeline = Pipeline([
        ('imputer_exp', SimpleImputer(strategy='median')), # Étape 1: Remplace les NaN par la médiane des années d'expérience.
        ('scaler_exp', StandardScaler())                   # Étape 2: Normalise les années d'expérience.
    ])

    # --- 3.c. Pipeline pour les colonnes catégorielles ---
    # Séquence de traitements pour toutes les colonnes identifiées comme catégorielles.
    categorical_pipeline = Pipeline([
        ('imputer_cat', SimpleImputer(strategy='most_frequent')), # Étape 1: Remplace les NaN par la valeur la plus fréquente de chaque colonne catégorielle.
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop=None))
      
    ])

    # --- 4. SÉLECTION DYNAMIQUE DES COLONNES POUR LES TRANSFORMATEURS ---
    # On vérifie si les colonnes 'Age' et 'Annees_experience' sont bien présentes dans X_train avant de leur appliquer un pipeline.
    # Cela rend le code plus robuste si ces colonnes venaient à manquer.
    age_col_for_transformer = [col for col in ['Age'] if col in X_train.columns]
    experience_col_for_transformer = [col for col in ['Annees_experience'] if col in X_train.columns]

    # --- 5. CONSTRUCTION DE LA LISTE DES TRANSFORMATEURS POUR ColumnTransformer ---
    # On prépare une liste qui contiendra les instructions pour le ColumnTransformer :
    # quel pipeline appliquer à quelles colonnes.
    transformers_for_columntransformer = []

    if age_col_for_transformer: 
        transformers_for_columntransformer.append(
            ('age_processing', age_pipeline, age_col_for_transformer)
        )
    if experience_col_for_transformer: 
        transformers_for_columntransformer.append(
            ('experience_processing', experience_pipeline, experience_col_for_transformer)
        )
    if categorical_cols: 
        transformers_for_columntransformer.append(
            ('categorical_processing', categorical_pipeline, categorical_cols)
        )

    # --- 6. CRÉATION DU ColumnTransformer (PREPROCESSOR) ---
    # Le ColumnTransformer est l'outil qui va orchestrer l'application des différents pipelines aux bonnes colonnes.
    if transformers_for_columntransformer: 
        preprocessor = ColumnTransformer(
            transformers=transformers_for_columntransformer, 
            remainder='drop' 
                            
        )
        print("Preprocessor ColumnTransformer créé avec succès.")

        # --- 7. CONFIGURATION DE L'AFFICHAGE DU PREPROCESSOR ---
        # Pour visualiser la structure du preprocessor sous forme de diagramme (très pratique !).
        from sklearn import set_config
        set_config(display='diagram') 
        display(preprocessor)        
        set_config(display='text')   
    else:
        print("ATTENTION : Aucun transformateur n'a été ajouté au ColumnTransformer. 'preprocessor' n'est pas défini.")
        preprocessor = None # On définit preprocessor à None pour éviter des erreurs plus tard.

else:
    print("ATTENTION : X_train ou les listes de colonnes numerical_cols/categorical_cols ne sont pas définis. Veuillez exécuter les cellules précédentes.")

Preprocessor ColumnTransformer créé avec succès.


ColumnTransformer(transformers=[('age_processing',
                                 Pipeline(steps=[('imputer_age',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler_age',
                                                  StandardScaler())]),
                                 ['Age']),
                                ('experience_processing',
                                 Pipeline(steps=[('imputer_exp',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler_exp',
                                                  StandardScaler())]),
                                 ['Annees_experience']),
                                ('categorical_processing',
                                 Pipeline(steps=[('imputer_cat',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['Sexe', 'Milieu', 'Region_geographique',
                                  'Etat_matrimonial', 'Niveau_education', 'CSP',
                                  'Secteur_emploi', 'Propriete_immobiliere',
                                  'Vehicule_motorise', 'Terrain_agricole',
                                  'Revenu_secondaire'])])

## 2.5 Application du Preprocessor et Récupération des N`sparse_output=False` dans `OneHotEncoder` :** C'est bien pour l'inspection.oms de Features

Le `preprocessor` étant défini, nous allons maintenant l'ajuster (`fit`) sur l'ensemble Si vous rencontrez des problèmes de mémoire plus tard avec un grand nombre de catégories après OHE, vous pourriez envisager de le d'entraînement `X_train` et l'utiliser pour transformer (`transform`) à la fois `X_train` et `X_test`.

Il est important de récupérer les noms des nouvelles features générées, notamment par le `OneHotEncoder passer à `True` et de travailler avec des matrices sparse (la plupart des modèles `sklearn` les acceptent). Pour `, pour pouvoir créer des DataFrames à partir des arrays NumPy retournés et inspecter les données transformées.

In [119]:
# --- 1. VÉRIFICATION DE LA DISPONIBILITÉ DES ÉLÉMENTS NÉCESSAIRES ---
if ('preprocessor' in locals() and preprocessor is not None and
    'X_train' in locals() and 'X_test' in locals() and
    'y_train' in locals() and 'y_test' in locals() and 
    'categorical_cols' in locals() and
    'age_col_for_transformer' in locals() and
    'experience_col_for_transformer' in locals()):

    # --- 2. APPLICATION DU PREPROCESSOR AUX DONNÉES ---
    print("Application du preprocessor sur X_train (fit_transform)...")
    # Le preprocessor apprend les transformations sur X_train et l'applique.
    X_train_processed_np = preprocessor.fit_transform(X_train)

    print("Application du preprocessor sur X_test (transform)...")
    # Le preprocessor applique les transformations apprises sur X_train à X_test.
    X_test_processed_np = preprocessor.transform(X_test)

    # --- 3. RÉCUPÉRATION DES NOMS DES COLONNES APRÈS TRANSFORMATION ---

    # --- 3.a. Noms pour les colonnes numériques traitées (si présentes dans le preprocessor) ---
    numeric_feature_names_processed = []
    # On vérifie si les transformateurs nommés 'age_processing' et 'experience_processing'
    if 'age_processing' in preprocessor.named_transformers_ and age_col_for_transformer:

        numeric_feature_names_processed.extend([f"{col}" for col in age_col_for_transformer])

    if 'experience_processing' in preprocessor.named_transformers_ and experience_col_for_transformer:
        numeric_feature_names_processed.extend([f"{col}" for col in experience_col_for_transformer]) 

    # --- 3.b. Noms pour les colonnes issues du OneHotEncoder (si présent) ---
    onehot_categorical_feature_names = [] 
    if 'categorical_processing' in preprocessor.named_transformers_ and categorical_cols:
        try:
            onehot_transformer = preprocessor.named_transformers_['categorical_processing']
            onehot_encoder_step = onehot_transformer.named_steps['onehot']
            onehot_categorical_feature_names = list(onehot_encoder_step.get_feature_names_out(categorical_cols))
        except KeyError as e:
            print(f"AVERTISSEMENT: Erreur de clé lors de la récupération des noms de features du OneHotEncoder: {e}")
            print("  Vérifiez les noms des étapes ('categorical_processing', 'onehot') dans votre ColumnTransformer.")
        except Exception as e:
            print(f"AVERTISSEMENT: Erreur générale lors de la récupération des noms de features du OneHotEncoder : {e}")

    # --- 3.c. Combinaison de tous les noms de features ---

    final_feature_names = numeric_feature_names_processed + onehot_categorical_feature_names

    # --- 4. RECONSTRUCTION DES DATAFRAMES AVEC LES NOUVEAUX NOMS DE COLONNES ---
    if len(final_feature_names) == X_train_processed_np.shape[1]:
        X_train_processed_df = pd.DataFrame(X_train_processed_np,
                                            columns=final_feature_names,
                                            index=X_train.index)

        X_test_processed_df = pd.DataFrame(X_test_processed_np,
                                           columns=final_feature_names,
                                           index=X_test.index)

        # --- 5. AFFICHAGE ET VÉRIFICATION DES DATAFRAMES TRAITÉES ---
        print(f"\nDimensions de X_train_processed_df : {X_train_processed_df.shape}")
        print("Aperçu de X_train_processed_df (5 premières lignes) :")
        display(X_train_processed_df.head())

        print(f"\nDimensions de X_test_processed_df : {X_test_processed_df.shape}")


        # Vérification finale des valeurs manquantes.
        print(f"\nNombre de NaN dans X_train_processed_df après traitement : {X_train_processed_df.isnull().sum().sum()}")
        print(f"Nombre de NaN dans X_test_processed_df après traitement : {X_test_processed_df.isnull().sum().sum()}")



    else:
        print("\nATTENTION : Le nombre de noms de colonnes récupérés ne correspond pas au nombre de colonnes dans les données traitées.")
        print(f"  Colonnes dans X_train_processed_np: {X_train_processed_np.shape[1]}")
        print(f"  Nombre de noms de features récupérés: {len(final_feature_names)}")
        print(f"  Noms de features récupérés: {final_feature_names}")
        print("  Les DataFrames X_train_processed_df et X_test_processed_df n'ont pas été créés correctement.")
        X_train_processed_df = pd.DataFrame(X_train_processed_np, index=X_train.index)
        X_test_processed_df = pd.DataFrame(X_test_processed_np, index=X_test.index)
        print("  DataFrames créés sans noms de colonnes pour inspection:")
        display(X_train_processed_df.head())


else:
    print("ATTENTION : Des variables nécessaires (preprocessor, X_train, X_test, y_train, y_test, categorical_cols, etc.) ne sont pas définies. Veuillez exécuter les cellules précédentes.")

Application du preprocessor sur X_train (fit_transform)...
Application du preprocessor sur X_test (transform)...

Dimensions de X_train_processed_df : (28000, 36)
Aperçu de X_train_processed_df (5 premières lignes) :


,Age,Annees_experience,Sexe_Femme,Sexe_Homme,Milieu_Rural,Milieu_Urbain,Region_geographique_Centre,Region_geographique_Est,Region_geographique_Nord,Region_geographique_Ouest,Region_geographique_Sud,Etat_matrimonial_Célibataire,Etat_matrimonial_Divorcé,Etat_matrimonial_Marié,Etat_matrimonial_Veuf,Niveau_education_Fondamental,Niveau_education_Sans niveau,Niveau_education_Secondaire,Niveau_education_Supérieur,CSP_Agriculteurs,CSP_Cadres supérieurs,CSP_Employés,CSP_Inactifs,CSP_Ouvriers,CSP_Professions intermédiaires,Secteur_emploi_Informel,Secteur_emploi_Privé,Secteur_emploi_Public,Propriete_immobiliere_Non,Propriete_immobiliere_Oui,Vehicule_motorise_Non,Vehicule_motorise_Oui,Terrain_agricole_Non,Terrain_agricole_Oui,Revenu_secondaire_Non,Revenu_secondaire_Oui
38015,-0.481264,-0.847681,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
2281,-1.445720,-0.847681,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
36629,1.150893,0.879334,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
6087,-1.148964,-0.847681,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
11792,-0.407075,-0.339736,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0



Dimensions de X_test_processed_df : (12000, 36)

Nombre de NaN dans X_train_processed_df après traitement : 0
Nombre de NaN dans X_test_processed_df après traitement : 0


# 3. Création et Validation des Modèles

Après avoir préparé nos données, nous passons maintenant à l'étape de modélisation. L'objectif est de :
1.  Sélectionner plusieurs algorithmes de régression.
2.  Ajuster (fine-tune) leurs hyperparamètres en utilisant la technique de validation croisée sur l'ensemble d'entraînement (`X_train_processed_df`, `y_train`).
3.  Comparer les performances des différents modèles sur la base des métriques MAE, RMSE, et R².
4.  Choisir le modèle le plus performant.

Les modèles à considérer sont : Régression Linéaire, Arbres de Décision, Forêts d'Arbres Décisionnels, Gradient Boosting, et Réseaux de Neurones Multi-couches (MLPRegressor).

## 3.1 Modèle 1 : Régression Linéaire (`LinearRegression`)

Nous commençons notre exploration des modèles de régression avec un algorithme fondamental : la Régression Linéaire. Ce modèle cherche à établir une relation linéaire entre les variables explicatives (features) et la variable cible (revenu annuel).

Bien que la Régression Linéaire soit simple et interprétable, ses performances peuvent être limitées si les relations sous-jacentes dans les données ne sont pas linéaires ou si les hypothèses du modèle (comme l'absence de forte multicollinéarité ou l'homoscédasticité des erreurs) ne sont pas respectées.

Pour ce modèle, le cahier des charges indique qu'il n'y a **aucun hyperparamètre à ajuster** via `GridSearchCV`. Nous allons donc l'entraîner directement sur les données d'apprentissage et évaluer ses performances initiales en utilisant la validation croisée pour obtenir une estimation plus robuste des métriques MAE, RMSE et R².

In [94]:

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

model_performance = {} 
RANDOM_STATE = 42

if ('X_train_processed_df' in locals() and 'y_train' in locals() and
    'X_test_processed_df' in locals() and 'y_test' in locals()):

    print("--- Modèle : Régression Linéaire (Évaluation sur X_test) ---")

    # 1. Initialisation du modèle
    linear_model = LinearRegression()

    # 2. Entraînement du modèle sur l'ensemble d'entraînement complet prétraité
    print("\nEntraînement du modèle sur X_train_processed_df...")
    linear_model.fit(X_train_processed_df, y_train)

    # 3. Prédictions sur l'ensemble de test prétraité
    print("\nPrédictions sur X_test_processed_df...")
    predictions_test = linear_model.predict(X_test_processed_df)

    # 4. Évaluation des performances sur l'ensemble de test
    mae_lr = mean_absolute_error(y_test, predictions_test)
    rmse_lr = np.sqrt(mean_squared_error(y_test, predictions_test))
    r2_lr = r2_score(y_test, predictions_test)

    print("\n--- Évaluation du Modèle sur l'Ensemble de Test (X_test) ---")
    print(f"  MAE (Mean Absolute Error)        : {mae_lr:.2f} DH")
    print(f"  RMSE (Root Mean Squared Error)   : {rmse_lr:.2f} DH")
    print(f"  R² (Coefficient de détermination): {r2_lr:.4f}")

    # Stocker les performances
    model_performance['Linear Regression'] = {
        'MAE': mae_lr,
        'RMSE': rmse_lr,
        'R2': r2_lr,
        'Best Params': 'N/A'
    }

    # Afficher un aperçu des prédictions vs valeurs réelles pour vérification
    print("\n--- Aperçu des Prédictions vs Valeurs Réelles (sur X_test) ---")
    df_predictions = pd.DataFrame({'Valeur_Réelle': y_test.values, 
                                   'Valeur_Prédite': predictions_test})
    display(df_predictions.head(10)) 
else:
    print("ATTENTION : Les données prétraitées (X_train_processed_df, y_train, X_test_processed_df, y_test) ne sont pas définies.")
    print("Veuillez exécuter les cellules de prétraitement des données (jusqu'à 2.5 inclus avec les modifications appropriées).")

--- Modèle : Régression Linéaire (Évaluation sur X_test) ---

Entraînement du modèle sur X_train_processed_df...

Prédictions sur X_test_processed_df...

--- Évaluation du Modèle sur l'Ensemble de Test (X_test) ---
  MAE (Mean Absolute Error)        : 8112.47 DH
  RMSE (Root Mean Squared Error)   : 17097.94 DH
  R² (Coefficient de détermination): 0.7394

--- Aperçu des Prédictions vs Valeurs Réelles (sur X_test) ---


,Valeur_Réelle,Valeur_Prédite
0,300.000000,-4420.060133
1,9514.349720,9532.598402
2,13886.129297,30755.694900
3,12291.185637,12126.760540
4,10524.116430,13347.567358
5,20478.219738,20639.733890
6,6465.972785,22327.635116
7,29689.598439,26558.987518
8,9097.518019,18506.481355
9,107554.335002,111933.826843


## 3.2. Arbres de Décision (DecisionTreeRegressor)

Nous allons maintenant entraîner un modèle d'Arbre de Décision. Pour optimiser ses performances, nous utiliserons `GridSearchCV` pour trouver la meilleure combinaison d'hyperparamètres parmi celles spécifiées. Les hyperparamètres que nous allons tester sont :
*   `criterion` : La fonction pour mesurer la qualité d'une division.
*   `max_depth` : La profondeur maximale de l'arbre.
*   `min_samples_split` : Le nombre minimum d'échantillons requis pour diviser un nœud interne.

L'objectif est de trouver un arbre qui généralise bien sans surapprendre les données d'entraînement.

In [95]:
if ('X_train_processed_df' in locals() and 'y_train' in locals() and
    'X_test_processed_df' in locals() and 'y_test' in locals()):

    print("--- Modèle : Arbre de Décision (DecisionTreeRegressor) ---")

    # Définir la grille d'hyperparamètres
    param_grid_dt = {
        'criterion': ['squared_error', 'friedman_mse', 'absolute_error'],
        'max_depth': [5, 10, 15, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    # Note: 'absolute_error' peut être significativement plus lent.

    # Initialiser GridSearchCV
    dt_regressor = DecisionTreeRegressor(random_state=RANDOM_SEED)
    grid_search_dt = GridSearchCV(
        estimator=dt_regressor,
        param_grid=param_grid_dt,
        cv=5, # Nombre de plis pour la validation croisée
        scoring='r2', # Métrique pour l'évaluation, R² est pertinent ici
        n_jobs=-1, # Utiliser tous les processeurs disponibles
        verbose=1 # Afficher des messages pendant l'ajustement
    )

    print(f"\nAjustement des hyperparamètres pour DecisionTreeRegressor en cours (CV={grid_search_dt.cv})...")
    grid_search_dt.fit(X_train_processed_df, y_train)

    # Meilleurs hyperparamètres et meilleur estimateur
    print(f"\nMeilleurs hyperparamètres trouvés pour DecisionTreeRegressor : {grid_search_dt.best_params_}")
    best_dt_model = grid_search_dt.best_estimator_

    # Prédictions sur l'ensemble de test avec le meilleur modèle
    print("\nPrédictions sur X_test_processed_df avec le meilleur DecisionTreeRegressor...")
    predictions_dt = best_dt_model.predict(X_test_processed_df)

    # Évaluation des performances
    mae_dt = mean_absolute_error(y_test, predictions_dt)
    rmse_dt = np.sqrt(mean_squared_error(y_test, predictions_dt))
    r2_dt = r2_score(y_test, predictions_dt)

    print("\n--- Évaluation du Meilleur DecisionTreeRegressor sur l'Ensemble de Test ---")
    print(f"  MAE (Mean Absolute Error)        : {mae_dt:.2f} DH")
    print(f"  RMSE (Root Mean Squared Error)   : {rmse_dt:.2f} DH")
    print(f"  R² (Coefficient de détermination): {r2_dt:.4f}")

    # Stocker les performances
    if 'model_performance' in locals():
        model_performance['Decision Tree'] = {
            'MAE': mae_dt,
            'RMSE': rmse_dt,
            'R2': r2_dt,
            'Best Params': grid_search_dt.best_params_
        }
        print("\nPerformances de l'Arbre de Décision stockées.")

    # Afficher un aperçu des prédictions vs valeurs réelles
    print("\n--- Aperçu des Prédictions vs Valeurs Réelles (Decision Tree) ---")
    df_predictions_dt = pd.DataFrame({'Valeur_Réelle': y_test.values, 
                                      'Valeur_Prédite_DT': predictions_dt})
    display(df_predictions_dt.head(10))

else:
    print("ATTENTION : Les données prétraitées ne sont pas définies. Veuillez exécuter les cellules précédentes.")


--- Modèle : Arbre de Décision (DecisionTreeRegressor) ---

Ajustement des hyperparamètres pour DecisionTreeRegressor en cours (CV=5)...
Fitting 5 folds for each of 108 candidates, totalling 540 fits


KeyboardInterrupt: 

## 3.2. Arbres de Décision (DecisionTreeRegressor)

Nous allons maintenant entraîner un modèle d'Arbre de Décision. Pour optimiser ses performances, nous utiliserons `GridSearchCV` pour trouver la meilleure combinaison d'hyperparamètres parmi celles spécifiées. Les hyperparamètres que nous allons tester sont :
*   `criterion` : La fonction pour mesurer la qualité d'une division.
*   `max_depth` : La profondeur maximale de l'arbre.
*   `min_samples_split` : Le nombre minimum d'échantillons requis pour diviser un nœud interne.

L'objectif est de trouver un arbre qui généralise bien sans surapprendre les données d'entraînement.

In [ ]:
if ('X_train_processed_df' in locals() and 'y_train' in locals() and
    'X_test_processed_df' in locals() and 'y_test' in locals()):

    print("--- Modèle : Arbre de Décision (DecisionTreeRegressor) ---")

    # Définir la grille d'hyperparamètres
    param_grid_dt = {
        'criterion': ['squared_error', 'friedman_mse', 'absolute_error'],
        'max_depth': [5, 10, 15, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    # Note: 'absolute_error' peut être significativement plus lent.

    # Initialiser GridSearchCV
    dt_regressor = DecisionTreeRegressor(random_state=RANDOM_SEED)
    grid_search_dt = GridSearchCV(
        estimator=dt_regressor,
        param_grid=param_grid_dt,
        cv=5, # Nombre de plis pour la validation croisée
        scoring='r2', # Métrique pour l'évaluation, R² est pertinent ici
        n_jobs=-1, # Utiliser tous les processeurs disponibles
        verbose=1 # Afficher des messages pendant l'ajustement
    )

    print(f"\nAjustement des hyperparamètres pour DecisionTreeRegressor en cours (CV={grid_search_dt.cv})...")
    grid_search_dt.fit(X_train_processed_df, y_train)

    # Meilleurs hyperparamètres et meilleur estimateur
    print(f"\nMeilleurs hyperparamètres trouvés pour DecisionTreeRegressor : {grid_search_dt.best_params_}")
    best_dt_model = grid_search_dt.best_estimator_

    # Prédictions sur l'ensemble de test avec le meilleur modèle
    print("\nPrédictions sur X_test_processed_df avec le meilleur DecisionTreeRegressor...")
    predictions_dt = best_dt_model.predict(X_test_processed_df)

    # Évaluation des performances
    mae_dt = mean_absolute_error(y_test, predictions_dt)
    rmse_dt = np.sqrt(mean_squared_error(y_test, predictions_dt))
    r2_dt = r2_score(y_test, predictions_dt)

    print("\n--- Évaluation du Meilleur DecisionTreeRegressor sur l'Ensemble de Test ---")
    print(f"  MAE (Mean Absolute Error)        : {mae_dt:.2f} DH")
    print(f"  RMSE (Root Mean Squared Error)   : {rmse_dt:.2f} DH")
    print(f"  R² (Coefficient de détermination): {r2_dt:.4f}")

    # Stocker les performances
    if 'model_performance' in locals():
        model_performance['Decision Tree'] = {
            'MAE': mae_dt,
            'RMSE': rmse_dt,
            'R2': r2_dt,
            'Best Params': grid_search_dt.best_params_
        }
        print("\nPerformances de l'Arbre de Décision stockées.")

    # Afficher un aperçu des prédictions vs valeurs réelles
    print("\n--- Aperçu des Prédictions vs Valeurs Réelles (Decision Tree) ---")
    df_predictions_dt = pd.DataFrame({'Valeur_Réelle': y_test.values, 
                                      'Valeur_Prédite_DT': predictions_dt})
    display(df_predictions_dt.head(10))

else:
    print("ATTENTION : Les données prétraitées ne sont pas définies. Veuillez exécuter les cellules précédentes.")


## 3.3. Forêts Aléatoires (RandomForestRegressor)

Les Forêts Aléatoires sont des modèles d'ensemble qui construisent plusieurs arbres de décision lors de l'entraînement et produisent la moyenne des prédictions de chaque arbre. Elles sont généralement plus robustes et performantes que les arbres de décision uniques, et moins sujettes au surapprentissage.
Nous utiliserons `GridSearchCV` pour optimiser les hyperparamètres suivants :
*   `n_estimators` : Le nombre d'arbres dans la forêt.
*   `max_depth` : La profondeur maximale de chaque arbre.
*   `min_samples_split` : Le nombre minimum d'échantillons requis pour diviser un nœud.
*   `min_samples_leaf` : Le nombre minimum d'échantillons requis à un nœud feuille.

In [ ]:
if ('X_train_processed_df' in locals() and 'y_train' in locals() and
    'X_test_processed_df' in locals() and 'y_test' in locals()):

    print("--- Modèle : Forêt Aléatoire (RandomForestRegressor) ---")

    # Définir la grille d'hyperparamètres (peut être réduite pour un calcul plus rapide)
    param_grid_rf = {
        'n_estimators': [100, 150], # Nombre d'arbres
        'max_depth': [10, 20, None],      # Profondeur max des arbres
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }
    # Pour un test rapide, vous pouvez réduire le nombre de valeurs par hyperparamètre.
    # Exemple : 'n_estimators': [50], 'max_depth': [10]

    # Initialiser GridSearchCV
    rf_regressor = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1) # n_jobs=-1 pour RandomForest aussi
    grid_search_rf = GridSearchCV(
        estimator=rf_regressor,
        param_grid=param_grid_rf,
        cv=3, # Réduire CV pour accélérer si nécessaire (5 est une bonne pratique)
        scoring='r2',
        n_jobs=-1, # Utiliser tous les processeurs pour GridSearchCV
        verbose=1
    )

    print(f"\nAjustement des hyperparamètres pour RandomForestRegressor en cours (CV={grid_search_rf.cv})...")
    grid_search_rf.fit(X_train_processed_df, y_train)

    # Meilleurs hyperparamètres et meilleur estimateur
    print(f"\nMeilleurs hyperparamètres trouvés pour RandomForestRegressor : {grid_search_rf.best_params_}")
    best_rf_model = grid_search_rf.best_estimator_

    # Prédictions sur l'ensemble de test
    print("\nPrédictions sur X_test_processed_df avec le meilleur RandomForestRegressor...")
    predictions_rf = best_rf_model.predict(X_test_processed_df)

    # Évaluation des performances
    mae_rf = mean_absolute_error(y_test, predictions_rf)
    rmse_rf = np.sqrt(mean_squared_error(y_test, predictions_rf))
    r2_rf = r2_score(y_test, predictions_rf)

    print("\n--- Évaluation du Meilleur RandomForestRegressor sur l'Ensemble de Test ---")
    print(f"  MAE (Mean Absolute Error)        : {mae_rf:.2f} DH")
    print(f"  RMSE (Root Mean Squared Error)   : {rmse_rf:.2f} DH")
    print(f"  R² (Coefficient de détermination): {r2_rf:.4f}")

    # Stocker les performances
    if 'model_performance' in locals():
        model_performance['Random Forest'] = {
            'MAE': mae_rf,
            'RMSE': rmse_rf,
            'R2': r2_rf,
            'Best Params': grid_search_rf.best_params_
        }
        print("\nPerformances de la Forêt Aléatoire stockées.")

    # Afficher un aperçu des prédictions vs valeurs réelles
    print("\n--- Aperçu des Prédictions vs Valeurs Réelles (Random Forest) ---")
    df_predictions_rf = pd.DataFrame({'Valeur_Réelle': y_test.values, 
                                      'Valeur_Prédite_RF': predictions_rf})
    display(df_predictions_rf.head(10))
else:
    print("ATTENTION : Les données prétraitées ne sont pas définies. Veuillez exécuter les cellules précédentes.")

## 3.4. Gradient Boosting Regressor 

Le Gradient Boosting est une autre technique d'ensemble puissante qui construit des modèles de manière séquentielle, où chaque nouveau modèle corrige les erreurs des précédents. Nous allons utiliser `GradientBoostingRegressor` de scikit-learn et optimiser ses hyperparamètres avec `GridSearchCV`.
Les hyperparamètres clés incluent :
*   `n_estimators` : Le nombre d'étapes de boosting (arbres).
*   `learning_rate` : Le taux d'apprentissage qui réduit la contribution de chaque arbre.
*   `max_depth` : La profondeur maximale des estimateurs de régression individuels.
*   `subsample` : La fraction d'échantillons à utiliser pour ajuster les arbres individuels.

In [ ]:
if ('X_train_processed_df' in locals() and 'y_train' in locals() and
    'X_test_processed_df' in locals() and 'y_test' in locals()):

    print("--- Modèle : Gradient Boosting Regressor ---")

    # Définir la grille d'hyperparamètres (peut être réduite pour un calcul plus rapide)
    param_grid_gbr = {
        'n_estimators': [100, 200],       # Nombre d'arbres
        'learning_rate': [0.05, 0.1],    # Taux d'apprentissage
        'max_depth': [3, 5],             # Profondeur max des arbres
        'subsample': [0.8, 1.0]          # Fraction d'échantillons pour l'entraînement de chaque arbre
    }
    # Pour un test rapide : 'n_estimators': [50], 'learning_rate': [0.1], 'max_depth': [3]

    # Initialiser GridSearchCV
    gbr_regressor = GradientBoostingRegressor(random_state=RANDOM_SEED)
    grid_search_gbr = GridSearchCV(
        estimator=gbr_regressor,
        param_grid=param_grid_gbr,
        cv=3, # Réduire CV pour accélérer si nécessaire
        scoring='r2',
        n_jobs=-1,
        verbose=1
    )

    print(f"\nAjustement des hyperparamètres pour GradientBoostingRegressor en cours (CV={grid_search_gbr.cv})...")
    grid_search_gbr.fit(X_train_processed_df, y_train)

    # Meilleurs hyperparamètres et meilleur estimateur
    print(f"\nMeilleurs hyperparamètres trouvés pour GradientBoostingRegressor : {grid_search_gbr.best_params_}")
    best_gbr_model = grid_search_gbr.best_estimator_

    # Prédictions sur l'ensemble de test
    print("\nPrédictions sur X_test_processed_df avec le meilleur GradientBoostingRegressor...")
    predictions_gbr = best_gbr_model.predict(X_test_processed_df)

    # Évaluation des performances
    mae_gbr = mean_absolute_error(y_test, predictions_gbr)
    rmse_gbr = np.sqrt(mean_squared_error(y_test, predictions_gbr))
    r2_gbr = r2_score(y_test, predictions_gbr)

    print("\n--- Évaluation du Meilleur GradientBoostingRegressor sur l'Ensemble de Test ---")
    print(f"  MAE (Mean Absolute Error)        : {mae_gbr:.2f} DH")
    print(f"  RMSE (Root Mean Squared Error)   : {rmse_gbr:.2f} DH")
    print(f"  R² (Coefficient de détermination): {r2_gbr:.4f}")

    # Stocker les performances
    if 'model_performance' in locals():
        model_performance['Gradient Boosting'] = {
            'MAE': mae_gbr,
            'RMSE': rmse_gbr,
            'R2': r2_gbr,
            'Best Params': grid_search_gbr.best_params_
        }
        print("\nPerformances du Gradient Boosting Regressor stockées.")

    # Afficher un aperçu des prédictions vs valeurs réelles
    print("\n--- Aperçu des Prédictions vs Valeurs Réelles (Gradient Boosting) ---")
    df_predictions_gbr = pd.DataFrame({'Valeur_Réelle': y_test.values, 
                                       'Valeur_Prédite_GBR': predictions_gbr})
    display(df_predictions_gbr.head(10))
else:
    print("ATTENTION : Les données prétraitées ne sont pas définies. Veuillez exécuter les cellules précédentes.")


## 3.5. Modèle : Réseau de Neurones (MLP Regressor)

Le Perceptron Multi-Couches (MLP) est un type de réseau de neurones artificiels feedforward. Il se compose d'au moins trois couches de nœuds : une couche d'entrée, une ou plusieurs couches cachées et une couche de sortie. Chaque nœud, à l'exception de ceux de la couche d'entrée, est un neurone qui utilise une fonction d'activation non linéaire. Le MLP utilise une technique d'apprentissage supervisé appelée rétropropagation pour l'entraînement. Sa capacité à apprendre des relations non linéaires complexes en fait un candidat intéressant pour notre problème de régression.

In [ ]:

if ('X_train_processed_df' in locals() and 'y_train' in locals() and
    'X_test_processed_df' in locals() and 'y_test' in locals()):

    print("--- Modèle : MLP Regressor (Réseau de Neurones) ---")

    # Définir la grille d'hyperparamètres pour MLPRegressor
    # Réduire la complexité pour un calcul plus rapide lors des tests initiaux
    param_grid_mlp = {
        'hidden_layer_sizes': [(50,), (100,), (100, 50), (100, 100)],
        'activation': ['relu'],
        'solver': ['adam'],
        'alpha': [0.0001, 0.001, 0.01],
        'learning_rate_init': [0.001, 0.01],
        'max_iter': [300, 500], # S'assurer que early_stopping est activé dans l'estimateur
        # 'batch_size': [32, 64] # Pourrait être testé dans une seconde passe
    }

    # Initialiser GridSearchCV
    grid_search_mlp = GridSearchCV(
        estimator=MLPRegressor(random_state=RANDOM_STATE, early_stopping=True, n_iter_no_change=15, tol=1e-4), # Ajuster n_iter_no_change et tol
        param_grid=param_grid_mlp,
        cv=2, # Maintenir cv=2 ou 3 pour la vitesse
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=1
    )

    print(f"\nAjustement des hyperparamètres pour MLPRegressor en cours (CV={grid_search_mlp.cv})...")
    grid_search_mlp.fit(X_train_processed_df, y_train)

    # Meilleurs hyperparamètres et meilleur estimateur
    print(f"\nMeilleurs hyperparamètres trouvés pour MLPRegressor : {grid_search_mlp.best_params_}")
    best_mlp_model = grid_search_mlp.best_estimator_

    # Prédictions sur l'ensemble de test
    print("\nPrédictions sur X_test_processed_df avec le meilleur MLPRegressor...")
    predictions_mlp = best_mlp_model.predict(X_test_processed_df)

    # Évaluation des performances
    mae_mlp = mean_absolute_error(y_test, predictions_mlp)
    rmse_mlp = np.sqrt(mean_squared_error(y_test, predictions_mlp))
    r2_mlp = r2_score(y_test, predictions_mlp)

    print("\n--- Évaluation du Meilleur MLPRegressor sur l'Ensemble de Test ---")
    print(f"  MAE (Mean Absolute Error)        : {mae_mlp:.2f} DH")
    print(f"  RMSE (Root Mean Squared Error)   : {rmse_mlp:.2f} DH")
    print(f"  R² (Coefficient de détermination): {r2_mlp:.4f}")

    # Stocker les performances
    if 'model_performance' in locals():
        model_performance['MLP Regressor'] = {
            'MAE': mae_mlp,
            'RMSE': rmse_mlp,
            'R2': r2_mlp,
            'Best Params': grid_search_mlp.best_params_
        }
        print("\nPerformances du MLP Regressor stockées.")

    # Afficher un aperçu des prédictions vs valeurs réelles
    print("\n--- Aperçu des Prédictions vs Valeurs Réelles (MLP Regressor) ---")
    df_predictions_mlp = pd.DataFrame({'Valeur_Réelle': y_test.values.ravel(), # Assurez-vous que y_test est aussi 1D
                                       'Valeur_Prédite_MLP': predictions_mlp})
    display(df_predictions_mlp.head(10))

else:
    print("ATTENTION : Les données prétraitées (X_train_processed_df, y_train, etc.) ne sont pas définies. Veuillez exécuter les cellules précédentes.")


## 3.6. Comparaison des Modèles

Après avoir entraîné et évalué individuellement plusieurs modèles, nous allons maintenant comparer leurs performances sur la base des métriques R², MAE et RMSE. Cela nous aidera à sélectionner le modèle le plus performant pour notre tâche de prédiction de revenu.

In [ ]:
if 'model_performance' in locals() and model_performance:
    
    # Convertir le dictionnaire de performances en DataFrame pour une visualisation facile
    performance_df = pd.DataFrame.from_dict(model_performance, orient='index')
    performance_df = performance_df.sort_values(by='R2', ascending=False)  # Trier par R2

    print("--- Tableau Comparatif des Performances des Modèles ---")
    display(performance_df[['R2', 'MAE', 'RMSE']])

    # Visualisation des performances
    fig, axes = plt.subplots(3, 1, figsize=(12, 18))
    fig.suptitle('Comparaison des Performances des Modèles', fontsize=16)

    # R²
    sns.barplot(x=performance_df.index, y='R2', data=performance_df, ax=axes[0], palette='viridis')
    axes[0].set_title('Coefficient de Détermination (R²)')
    axes[0].set_ylabel('R² Score')
    axes[0].tick_params(axis='x', rotation=45)
    for i, v in enumerate(performance_df['R2']):
        axes[0].text(i, v + 0.01, f"{v:.4f}", color='black', ha='center')

    # MAE
    sns.barplot(x=performance_df.index, y='MAE', data=performance_df, ax=axes[1], palette='viridis')
    axes[1].set_title('Mean Absolute Error (MAE)')
    axes[1].set_ylabel('MAE (DH)')
    axes[1].tick_params(axis='x', rotation=45)
    for i, v in enumerate(performance_df['MAE']):
        axes[1].text(i, v + 0.01 * performance_df['MAE'].max(), f"{v:.2f}", color='black', ha='center')

    # RMSE
    sns.barplot(x=performance_df.index, y='RMSE', data=performance_df, ax=axes[2], palette='viridis')
    axes[2].set_title('Root Mean Squared Error (RMSE)')
    axes[2].set_ylabel('RMSE (DH)')
    axes[2].tick_params(axis='x', rotation=45)
    for i, v in enumerate(performance_df['RMSE']):
        axes[2].text(i, v + 0.01 * performance_df['RMSE'].max(), f"{v:.2f}", color='black', ha='center')

    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Ajuster pour le titre principal
    plt.show()

    # Sélection du meilleur modèle basé sur R2 (ou une combinaison de métriques)
    best_model_name = performance_df['R2'].idxmax()
    print(f"\n🏆 Le meilleur modèle basé sur le R² est : {best_model_name} avec R² = {performance_df.loc[best_model_name, 'R2']:.4f}")
    print(f"   MAE: {performance_df.loc[best_model_name, 'MAE']:.2f} DH")
    print(f"   RMSE: {performance_df.loc[best_model_name, 'RMSE']:.2f} DH")
    print(f"   Meilleurs hyperparamètres: {performance_df.loc[best_model_name, 'Best Params']}")

    # Stocker le nom du meilleur modèle et ses performances pour la sauvegarde
    if best_model_name == 'Linear Regression':
        final_best_model_object = linear_model
    elif best_model_name == 'Decision Tree':
        final_best_model_object = best_dt_model
    elif best_model_name == 'Random Forest':
        final_best_model_object = best_rf_model
    elif best_model_name == 'Gradient Boosting':
        final_best_model_object = best_gbr_model
    elif best_model_name == 'MLP Regressor':
        final_best_model_object = best_mlp_model
    else:
        final_best_model_object = None
        print(f"Attention : L'objet du modèle pour '{best_model_name}' n'a pas été assigné à 'final_best_model_object'.")

else:
    print("ATTENTION : Le dictionnaire 'model_performance' n'est pas défini ou est vide. Veuillez exécuter les cellules d'entraînement des modèles.")


--- Tableau Comparatif des Performances des Modèles ---


,R2,MAE,RMSE
Linear Regression,0.739432,8112.471233,17097.938719



🏆 Le meilleur modèle basé sur le R² est : Linear Regression avec R² = 0.7394
   MAE: 8112.47 DH
   RMSE: 17097.94 DH
   Meilleurs hyperparamètres: N/A


# 4. Sauvegarde du Meilleur Modèle et des Composants de Prétraitement

Une fois le meilleur modèle identifié, il est crucial de le sauvegarder, ainsi que tous les composants de prétraitement (imputers, scaler, liste des colonnes, etc.) nécessaires pour traiter de nouvelles données de la même manière que les données d'entraînement. Cela permettra de déployer le modèle et de faire des prédictions sur de nouvelles instances sans avoir à ré-entraîner ou à redéfinir le pipeline de prétraitement.

Nous allons sauvegarder :
1.  L'objet du modèle entraîné.
2.  Les composants du `ColumnTransformer` (imputers numériques et catégoriels, scaler).
3.  La liste des colonnes après l'encodage one-hot (`train_cols_after_dummies`), qui est essentielle pour s'assurer que les nouvelles données ont la même structure.
4.  Les listes des colonnes numériques et catégorielles originales.
5.  Les métadonnées de performance du modèle.

In [ ]:
import joblib
import os

if 'final_best_model_object' in locals() and final_best_model_object is not None and \
   'preprocessor' in locals() and \
   'model_performance' in locals() and best_model_name in model_performance and \
   'X_train_processed_df' in locals() and \
   'numerical_cols' in locals() and 'categorical_cols' in locals():

    # Nom du fichier pour le modèle sauvegardé
    # Utiliser le nom du meilleur modèle pour le fichier pkl
    model_filename = f"best_{best_model_name.lower().replace(' ', '_')}_model.pkl"
    if best_model_name == 'MLP Regressor': # Pour correspondre à app.py et api.py
        model_filename = "best_mlp_model.pkl"


    # Créer un dictionnaire pour stocker tous les composants
    model_to_save = {
        'model': final_best_model_object,
        'preprocessor_components': {
            'num_imputer': preprocessor.named_transformers_['num_pipeline'].named_steps['imputer'],
            'cat_imputer': preprocessor.named_transformers_['cat_pipeline'].named_steps['imputer'],
            'scaler': preprocessor.named_transformers_['num_pipeline'].named_steps['scaler'],
            'train_cols_after_dummies': X_train_processed_df.columns.tolist(),
            'original_numerical_cols': numerical_cols,
            'original_categorical_cols': categorical_cols
        },
        'metadata': {
            'best_model_name': best_model_name,
            'performance': model_performance[best_model_name],
            'description': 'Modèle de prédiction de revenu marocain et ses composants de prétraitement.'
        }
    }

    # Sauvegarder le dictionnaire
    try:
        joblib.dump(model_to_save, model_filename)
        print(f"Modèle et composants sauvegardés avec succès dans '{model_filename}'")

        # Vérification du contenu (optionnel)
        loaded_model_data = joblib.load(model_filename)
        print("\nContenu du fichier sauvegardé (vérification) :")
        print(f"  Modèle: {type(loaded_model_data['model'])}")
        print(f"  Composants du préprocesseur: {list(loaded_model_data['preprocessor_components'].keys())}")
        print(f"  Métadonnées: {loaded_model_data['metadata']['best_model_name']}, R2: {loaded_model_data['metadata']['performance']['R2']:.4f}")

    except Exception as e:
        print(f"Erreur lors de la sauvegarde du modèle : {e}")
else:
    print("ATTENTION : Un ou plusieurs éléments requis pour la sauvegarde ne sont pas définis.")
    print("Vérifiez : final_best_model_object, preprocessor, model_performance, best_model_name, X_train_processed_df, numerical_cols, categorical_cols.")


# 5. Déploiement (Conceptuel)

Le déploiement est l'étape où le modèle de Machine Learning est mis à la disposition des utilisateurs finaux ou d'autres systèmes. Pour ce projet, nous avons envisagé deux approches principales pour le déploiement :

## 5.1. API Web avec FastAPI

FastAPI est un framework web moderne, rapide (haute performance), pour construire des APIs avec Python 3.7+ basé sur les indications de type Python standard.

**Principes :**
1.  **Créer un fichier `api.py`** : Ce fichier contiendra le code de l'application FastAPI.
2.  **Charger le modèle sauvegardé** : Au démarrage de l'application, le fichier `.pkl` contenant le modèle et les composants de prétraitement est chargé.
3.  **Définir un endpoint de prédiction** : Un endpoint (par exemple `/predict`) est créé pour recevoir les données d'entrée (caractéristiques de l'individu) au format JSON.
4.  **Prétraiter les données d'entrée** : Les données reçues sont prétraitées en utilisant les composants sauvegardés (imputation, encodage, scaling) pour les transformer dans le format attendu par le modèle.
5.  **Faire une prédiction** : Le modèle chargé est utilisé pour prédire le revenu.
6.  **Retourner la prédiction** : La prédiction est retournée, généralement au format JSON.

*Le fichier `api.py` fourni dans le projet implémente cette logique.*

## 5.2. Application Web Interactive avec Streamlit

Streamlit est un framework open-source permettant de créer et de partager rapidement de belles applications web pour des projets de data science et de machine learning, en utilisant uniquement Python.

**Principes :**
1.  **Créer un fichier `app.py`** : Ce fichier contiendra le code de l'application Streamlit.
2.  **Charger le modèle sauvegardé** : Similaire à FastAPI, le modèle et les composants sont chargés.
3.  **Créer une interface utilisateur** : Des widgets Streamlit (sliders, selectbox, number_input, etc.) sont utilisés pour permettre à l'utilisateur de saisir les caractéristiques.
4.  **Récupérer les entrées utilisateur**.
5.  **Prétraiter les données d'entrée** : Les données saisies sont collectées, formatées en DataFrame, puis prétraitées.
6.  **Faire une prédiction** : Le modèle effectue la prédiction.
7.  **Afficher le résultat** : Le revenu prédit est affiché à l'utilisateur, potentiellement avec des informations contextuelles ou des visualisations.
8.  **(Optionnel) Afficher les performances du modèle** : Les métriques du modèle sauvegardé peuvent également être affichées.

*Le fichier `app.py` fourni dans le projet implémente cette logique, permettant une prédiction locale si l'API n'est pas disponible.*

**Pour lancer les applications :**
*   **FastAPI** : `uvicorn api:app --reload` (depuis le terminal, dans le dossier du projet)
*   **Streamlit** : `streamlit run app.py` (depuis le terminal, dans le dossier du projet)

Ces deux approches permettent de rendre notre modèle de prédiction de revenu accessible et utilisable.